# LLM Embeddings + XGBoost Engagement Prediction Model

This notebook trains an XGBoost regression model to predict post engagement metrics using:
- **LLM embeddings** (Google Gemini) for post text semantic understanding
- **Persona metadata** (job role, affiliation, account age)
- **Context metadata** (audience size, baseline engagement, time window)

**Target Variables:**
- % positive reactions
- % negative reactions  
- Comment sentiment distribution
- Engagement velocity (early vs late)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import time
from typing import List, Dict, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# ML libraries
from xgboost import XGBRegressor, XGBClassifier
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, classification_report, confusion_matrix, roc_auc_score, accuracy_score
from sklearn.compose import ColumnTransformer

# Google AI for embeddings
try:
    import google.generativeai as genai
    GEMINI_AVAILABLE = True
except ImportError:
    print("Warning: google-generativeai not installed. Install with: pip install google-generativeai")
    GEMINI_AVAILABLE = False

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)

# Set random seed for reproducibility
np.random.seed(42)

## Configuration

Set up API keys and model parameters

In [ ]:
# Configuration
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY', '')
EMBEDDING_MODEL = 'models/text-embedding-004'  # or 'models/gemini-embedding-001'
DATA_PATH = 'data/'  # Path to your training data directory, or None to simulate

# Configure Gemini API if available
if GEMINI_AVAILABLE and GEMINI_API_KEY:
    genai.configure(api_key=GEMINI_API_KEY)
    print("✓ Gemini API configured")
else:
    print("⚠ Gemini API not configured. Embeddings will be simulated.")

## Synthetic Dataset Generation

Standalone functions to generate realistic synthetic datasets for Model A (individual-level) and Model B (post-level) with options to save to files.

In [ ]:
def save_dataset_to_file(df: pd.DataFrame, filepath: str, format: str = 'csv') -> None:
    """
    Save a dataset to file in specified format.
    
    Args:
        df: DataFrame to save
        filepath: Path to save file
        format: 'csv' or 'json'
    """
    if format.lower() == 'csv':
        df.to_csv(filepath, index=False)
        print(f"✓ Saved dataset to {filepath} (CSV, {len(df)} rows)")
    elif format.lower() == 'json':
        df.to_json(filepath, orient='records', indent=2)
        print(f"✓ Saved dataset to {filepath} (JSON, {len(df)} rows)")
    else:
        raise ValueError(f"Unsupported format: {format}. Use 'csv' or 'json'")

In [ ]:
def generate_comment_text(intent: str, role: str, post_topic: str, sentiment: float) -> str:
    """
    Generate realistic comment text based on intent, role, topic, and sentiment.
    """
    comment_templates = {
        'praise': [
            "Great post! {topic} is exactly what we need right now.",
            "Love this perspective on {topic}. Spot on!",
            "Excellent insights on {topic}. Thanks for sharing!",
            "This is why I follow {role}s like you. Great {topic} content!",
            "Absolutely agree. {topic} is crucial for the industry."
        ],
        'agreement': [
            "Completely agree. {topic} is a game-changer.",
            "This resonates. We've seen similar patterns with {topic}.",
            "Spot on. {topic} requires exactly this approach.",
            "Couldn't agree more. {topic} needs more attention.",
            "100% this. {topic} is the future."
        ],
        'question': [
            "Interesting take on {topic}. How do you handle [specific aspect]?",
            "Great post! Curious about your thoughts on [related topic]?",
            "Love this. Have you considered [alternative approach] for {topic}?",
            "This is helpful. What's your experience with [related issue]?",
            "Thanks for sharing. How does {topic} relate to [other topic]?"
        ],
        'criticism': [
            "I see your point, but {topic} also has [counterpoint].",
            "Interesting perspective, though I'd argue {topic} is more nuanced.",
            "Respectfully disagree. {topic} needs [different approach].",
            "I think {topic} requires considering [other factor].",
            "Good points, but {topic} isn't that simple."
        ],
        'insight_addition': [
            "Adding to this: {topic} also benefits from [additional insight].",
            "Great post! From my experience, {topic} works best when [insight].",
            "This is spot on. I'd add that {topic} requires [insight].",
            "Excellent points. Another angle: {topic} is enhanced by [insight].",
            "Love this. One more thing: {topic} is improved with [insight]."
        ],
        'none': ''
    }
    
    if intent == 'none' or intent not in comment_templates:
        return ''
    
    templates = comment_templates[intent]
    template = np.random.choice(templates)
    
    # Replace placeholders
    comment = template.format(topic=post_topic, role=role)
    
    # Adjust tone based on sentiment
    if sentiment < -0.3:
        comment = comment.replace("Great", "Interesting").replace("Love", "See")
    elif sentiment > 0.7:
        comment = comment.replace("Interesting", "Excellent").replace("See", "Love")
    
    return comment

In [ ]:
def calculate_role_post_alignment(role: str, post_topic: str, post_author_role: str) -> float:
    """
    Calculate alignment score between a role and a post (0-1).
    Higher score means more likely to engage.
    """
    alignment = 0.5  # Base alignment
    
    # Role-topic alignment
    tech_roles = ['Software Engineer', 'Data Scientist', 'CTO', 'Product Manager']
    leadership_roles = ['CEO', 'Founder', 'CTO']
    business_roles = ['CEO', 'Founder', 'Product Manager', 'Marketing Director']
    
    if post_topic in ['AI', 'technology', 'data science']:
        if role in tech_roles:
            alignment += 0.3
        if role == 'CTO':
            alignment += 0.2
    elif post_topic in ['startups', 'leadership']:
        if role in leadership_roles:
            alignment += 0.3
        if role == 'Founder':
            alignment += 0.2
    elif post_topic in ['productivity', 'innovation']:
        if role in business_roles:
            alignment += 0.2
    elif post_topic == 'marketing':
        if role == 'Marketing Director':
            alignment += 0.3
    
    # Author-role alignment (people engage more with similar roles)
    if role == post_author_role:
        alignment += 0.15
    
    # Seniority bonus (executives engage less frequently but more thoughtfully)
    if role in ['CEO', 'Founder', 'CTO']:
        alignment += 0.1  # But lower base reaction probability
    
    return min(alignment, 1.0)

In [ ]:
def generate_synthetic_model_a_data(
    n_posts: int = 100,
    interactions_per_post: int = 20,
    save_path: Optional[str] = None,
    seed: int = 42,
    realism_level: float = 0.8
) -> pd.DataFrame:
    """
    Generate realistic synthetic individual-level data (Model A format).
    
    Args:
        n_posts: Number of unique posts to generate
        interactions_per_post: Number of person interactions per post
        save_path: Optional path to save dataset (e.g., 'model_a_data.csv')
        seed: Random seed for reproducibility
        realism_level: 0-1, controls how realistic vs random (1 = most realistic)
    
    Returns:
        DataFrame with individual-level interaction data
    """
    np.random.seed(seed)
    
    print(f"Generating synthetic Model A data...")
    print(f"  Posts: {n_posts}")
    print(f"  Interactions per post: {interactions_per_post}")
    print(f"  Total interactions: {n_posts * interactions_per_post}")
    print(f"  Realism level: {realism_level}")
    
    # Post text templates
    post_templates = [
        "Excited to share my latest insights on {topic}! {opinion}",
        "Just finished reading an amazing article about {topic}. Here's what I learned: {insight}",
        "Hot take: {opinion} What do you think?",
        "After {experience}, I've realized that {insight}",
        "Breaking: {news} This changes everything!",
        "Quick thread on {topic}: 1/5 {point1}",
        "The future of {topic} is {prediction}. Here's why:",
        "I disagree with the common wisdom that {common_belief}. Instead, {alternative}",
        "5 lessons I learned from {experience}:",
        "Why {topic} matters more than you think: {reason}"
    ]
    
    topics = ["AI", "startups", "leadership", "technology", "productivity", "innovation", "data science", "marketing"]
    author_roles = ["Software Engineer", "Product Manager", "Data Scientist", "CEO", "CTO", "Marketing Director", "Founder"]
    time_windows = ["morning", "afternoon", "evening", "weekend"]
    
    # Role archetypes with metadata
    role_archetypes = {
        'Software Engineer': {'industry': 'Tech', 'seniority': 'Mid'},
        'Product Manager': {'industry': 'Tech', 'seniority': 'Mid'},
        'Data Scientist': {'industry': 'Tech', 'seniority': 'Mid'},
        'CEO': {'industry': 'Tech', 'seniority': 'Exec'},
        'CTO': {'industry': 'Tech', 'seniority': 'Exec'},
        'Founder': {'industry': 'Tech', 'seniority': 'Exec'},
        'Recruiter': {'industry': 'Tech', 'seniority': 'Mid'},
        'Marketing Director': {'industry': 'Tech', 'seniority': 'Senior'},
        'Designer': {'industry': 'Tech', 'seniority': 'Mid'},
        'Consultant': {'industry': 'Consulting', 'seniority': 'Senior'},
        'Student': {'industry': 'Education', 'seniority': 'Junior'}
    }
    
    reaction_types = ['like', 'love', 'insightful', 'celebrate', 'none']
    comment_intents = ['praise', 'agreement', 'question', 'criticism', 'insight_addition', 'none']
    
    # Generate posts
    posts = []
    for i in range(n_posts):
        template = np.random.choice(post_templates)
        topic = np.random.choice(topics)
        opinion = f"Game-changing innovation in {topic}"
        insight = f"{topic} is revolutionizing the industry"
        
        post_text = template.format(
            topic=topic,
            opinion=opinion,
            insight=insight,
            experience=f"working with {topic}",
            news=f"Major breakthrough in {topic}",
            point1=f"{topic} is evolving rapidly",
            prediction="bright",
            common_belief=f"{topic} is overhyped",
            alternative=f"{topic} is just getting started",
            reason=f"{topic} drives innovation"
        )
        
        author_role = np.random.choice(author_roles)
        time_posted = np.random.choice(time_windows)
        
        posts.append({
            'post_text': post_text,
            'post_topic': topic,
            'post_author_role': author_role,
            'time_posted': time_posted
        })
    
    # Generate individual interactions
    individual_data = []
    
    for post in posts:
        post_text = post['post_text']
        post_topic = post['post_topic']
        post_author_role = post['post_author_role']
        time_posted = post['time_posted']
        
        # Sample role archetypes for this post
        available_roles = list(role_archetypes.keys())
        sampled_roles = np.random.choice(
            available_roles,
            size=min(interactions_per_post, len(available_roles)),
            replace=False
        )
        
        for role in sampled_roles:
            role_info = role_archetypes[role]
            
            # Calculate role-post alignment
            alignment = calculate_role_post_alignment(role, post_topic, post_author_role)
            
            # Adjust probabilities based on realism level
            base_reaction_prob = 0.5 + (alignment - 0.5) * realism_level
            
            # Reaction type
            if np.random.random() < base_reaction_prob:
                # Weight reaction types based on alignment
                if alignment > 0.7:
                    reaction_weights = [0.3, 0.3, 0.25, 0.1, 0.05]  # More positive reactions
                elif alignment > 0.5:
                    reaction_weights = [0.4, 0.2, 0.2, 0.1, 0.1]
                else:
                    reaction_weights = [0.5, 0.1, 0.1, 0.05, 0.25]  # More 'none'
                
                reaction_type = np.random.choice(['like', 'love', 'insightful', 'celebrate', 'none'],
                                                p=reaction_weights)
            else:
                reaction_type = 'none'
            
            # Comment probability (correlated with reaction and alignment)
            if reaction_type == 'none':
                comment_prob = 0.01 * realism_level
            elif reaction_type in ['love', 'celebrate']:
                comment_prob = 0.25 * realism_level
            elif reaction_type == 'insightful':
                comment_prob = 0.35 * realism_level  # Insightful reactions often lead to comments
            else:
                comment_prob = 0.15 * realism_level
            
            # Executives comment less frequently but more thoughtfully
            if role in ['CEO', 'Founder', 'CTO']:
                comment_prob *= 0.6
            
            commented = np.random.random() < comment_prob
            
            # Comment sentiment and intent
            if commented:
                # Sentiment based on reaction type
                if reaction_type in ['love', 'celebrate']:
                    comment_sentiment = np.random.uniform(0.6, 1.0)
                    intent_weights = [0.5, 0.3, 0.1, 0.05, 0.05, 0.0]  # More praise/agreement
                elif reaction_type == 'insightful':
                    comment_sentiment = np.random.uniform(0.3, 0.8)
                    intent_weights = [0.1, 0.2, 0.2, 0.05, 0.4, 0.05]  # More insight addition
                elif reaction_type == 'like':
                    comment_sentiment = np.random.uniform(0.0, 0.7)
                    intent_weights = [0.2, 0.4, 0.2, 0.1, 0.05, 0.05]
                else:
                    comment_sentiment = np.random.uniform(-0.5, 0.5)
                    intent_weights = [0.05, 0.1, 0.3, 0.4, 0.1, 0.05]  # More questions/criticism
                
                # Adjust intent based on role
                if role in ['CEO', 'Founder']:
                    # Executives more likely to add insights or ask strategic questions
                    intent_weights = [0.1, 0.2, 0.3, 0.1, 0.25, 0.05]
                elif role in ['Software Engineer', 'Data Scientist']:
                    # Engineers more likely to ask technical questions
                    intent_weights = [0.15, 0.25, 0.35, 0.1, 0.1, 0.05]
                
                comment_intent = np.random.choice(comment_intents, p=intent_weights)
                comment_text = generate_comment_text(comment_intent, role, post_topic, comment_sentiment)
            else:
                comment_sentiment = 0.0
                comment_intent = 'none'
                comment_text = ''
            
            # Post metadata
            post_length = len(post_text.split())
            has_hashtags = '#' in post_text
            has_mentions = '@' in post_text
            has_urls = 'http' in post_text.lower()
            
            individual_data.append({
                # Post features
                'post_text': post_text,
                'post_topic': post_topic,
                'post_author_role': post_author_role,
                'post_length': post_length,
                'hashtags': has_hashtags,
                'mentions': has_mentions,
                'urls_present': has_urls,
                'time_posted': time_posted,
                
                # Person features
                'commenter_role': role,
                'commenter_industry': role_info['industry'],
                'commenter_seniority': role_info['seniority'],
                
                # Labels
                'reaction_type': reaction_type,
                'commented': int(commented),
                'comment_text': comment_text,
                'comment_sentiment': comment_sentiment,
                'comment_intent': comment_intent
            })
    
    df_individual = pd.DataFrame(individual_data)
    
    print(f"\\n✓ Generated {len(df_individual)} individual interactions")
    print(f"  Reactions: {df_individual['reaction_type'].value_counts().to_dict()}")
    print(f"  Comments: {df_individual['commented'].sum()} ({df_individual['commented'].mean():.1%})")
    
    # Save if path provided
    if save_path:
        save_dataset_to_file(df_individual, save_path)
    
    return df_individual

In [ ]:
def generate_synthetic_model_b_data(
    n_posts: int = 100,
    save_path: Optional[str] = None,
    seed: int = 42,
    realism_level: float = 0.8
) -> pd.DataFrame:
    """
    Generate realistic synthetic post-level data with aggregated features (Model B format).
    
    This simulates what Model A would produce when aggregated, plus post-level targets.
    
    Args:
        n_posts: Number of posts to generate
        save_path: Optional path to save dataset (e.g., 'model_b_data.csv')
        seed: Random seed for reproducibility
        realism_level: 0-1, controls how realistic vs random (1 = most realistic)
    
    Returns:
        DataFrame with post-level data including aggregated features and targets
    """
    np.random.seed(seed)
    
    print(f"Generating synthetic Model B data...")
    print(f"  Posts: {n_posts}")
    print(f"  Realism level: {realism_level}")
    
    # Post text templates (same as Model A)
    post_templates = [
        "Excited to share my latest insights on {topic}! {opinion}",
        "Just finished reading an amazing article about {topic}. Here's what I learned: {insight}",
        "Hot take: {opinion} What do you think?",
        "After {experience}, I've realized that {insight}",
        "Breaking: {news} This changes everything!",
        "Quick thread on {topic}: 1/5 {point1}",
        "The future of {topic} is {prediction}. Here's why:",
        "I disagree with the common wisdom that {common_belief}. Instead, {alternative}",
        "5 lessons I learned from {experience}:",
        "Why {topic} matters more than you think: {reason}"
    ]
    
    topics = ["AI", "startups", "leadership", "technology", "productivity", "innovation", "data science", "marketing"]
    author_roles = ["Software Engineer", "Product Manager", "Data Scientist", "CEO", "CTO", "Marketing Director", "Founder"]
    time_windows = ["morning", "afternoon", "evening", "weekend"]
    
    # Role archetypes for aggregation
    engineer_roles = ['Software Engineer', 'Data Scientist', 'CTO']
    pm_roles = ['Product Manager']
    founder_roles = ['Founder', 'CEO']
    all_roles = engineer_roles + pm_roles + founder_roles + ['Recruiter', 'Marketing Director', 'Designer', 'Consultant', 'Student']
    
    reaction_types = ['like', 'love', 'insightful', 'celebrate', 'none']
    
    post_data = []
    
    for i in range(n_posts):
        # Generate post
        template = np.random.choice(post_templates)
        topic = np.random.choice(topics)
        opinion = f"Game-changing innovation in {topic}"
        insight = f"{topic} is revolutionizing the industry"
        
        post_text = template.format(
            topic=topic,
            opinion=opinion,
            insight=insight,
            experience=f"working with {topic}",
            news=f"Major breakthrough in {topic}",
            point1=f"{topic} is evolving rapidly",
            prediction="bright",
            common_belief=f"{topic} is overhyped",
            alternative=f"{topic} is just getting started",
            reason=f"{topic} drives innovation"
        )
        
        author_role = np.random.choice(author_roles)
        time_posted = np.random.choice(time_windows)
        post_length = len(post_text.split())
        has_hashtags = '#' in post_text
        has_mentions = '@' in post_text
        has_urls = 'http' in post_text.lower()
        
        # Simulate aggregated Model A predictions
        # Base engagement depends on topic popularity and author credibility
        topic_popularity = 0.8 if topic in ['AI', 'technology', 'startups'] else 0.6
        author_credibility = 0.9 if author_role in ['CEO', 'Founder', 'CTO'] else 0.7
        
        base_engagement = (topic_popularity + author_credibility) / 2 * realism_level
        
        # Role engagement percentages (simulate what Model A would predict)
        # Engineers more likely to engage with tech posts
        if topic in ['AI', 'technology', 'data science']:
            engineer_engagement = base_engagement + np.random.uniform(0.1, 0.3) * realism_level
            pm_engagement = base_engagement + np.random.uniform(-0.1, 0.2) * realism_level
            founder_engagement = base_engagement + np.random.uniform(-0.1, 0.1) * realism_level
        elif topic in ['startups', 'leadership']:
            engineer_engagement = base_engagement + np.random.uniform(-0.1, 0.1) * realism_level
            pm_engagement = base_engagement + np.random.uniform(0.0, 0.2) * realism_level
            founder_engagement = base_engagement + np.random.uniform(0.2, 0.4) * realism_level
        else:
            engineer_engagement = base_engagement + np.random.uniform(-0.1, 0.1) * realism_level
            pm_engagement = base_engagement + np.random.uniform(-0.1, 0.1) * realism_level
            founder_engagement = base_engagement + np.random.uniform(-0.1, 0.1) * realism_level
        
        pct_engineers_reacting = np.clip(engineer_engagement, 0, 1)
        pct_pms_commenting = np.clip(pm_engagement * 0.3, 0, 1)  # Commenting is rarer
        pct_founders_engaging = np.clip(founder_engagement, 0, 1)
        
        # Reaction distribution
        total_people = np.random.randint(50, 500)  # Simulated audience size
        reactions_by_type = {}
        for rt in reaction_types:
            if rt == 'none':
                reactions_by_type[rt] = total_people * (1 - base_engagement)
            else:
                # Weight reactions based on engagement level
                if base_engagement > 0.7:
                    weights = {'like': 0.3, 'love': 0.3, 'insightful': 0.25, 'celebrate': 0.15}
                elif base_engagement > 0.5:
                    weights = {'like': 0.4, 'love': 0.2, 'insightful': 0.2, 'celebrate': 0.1}
                else:
                    weights = {'like': 0.5, 'love': 0.1, 'insightful': 0.1, 'celebrate': 0.05}
                
                reactions_by_type[rt] = total_people * base_engagement * weights.get(rt, 0.1)
        
        # Normalize reaction distribution
        total_reactions = sum(reactions_by_type.values())
        reaction_distribution = {k: v / total_reactions if total_reactions > 0 else 0 
                                for k, v in reactions_by_type.items()}
        
        # Comment volume (correlated with engagement)
        comment_volume = total_people * base_engagement * np.random.uniform(0.05, 0.25) * realism_level
        
        # Average sentiment by role (simulate)
        avg_sentiment_engineers = np.random.uniform(0.2, 0.8) if pct_engineers_reacting > 0.3 else 0.0
        avg_sentiment_pms = np.random.uniform(0.3, 0.9) if pct_pms_commenting > 0.2 else 0.0
        avg_sentiment_founders = np.random.uniform(0.4, 0.9) if pct_founders_engaging > 0.3 else 0.0
        
        # Calculate targets
        total_reactions_count = int(total_people * base_engagement)
        comment_volume_count = int(comment_volume)
        
        # Role distribution (percentage of each role in audience)
        role_dist = {
            'engineer_pct': np.random.uniform(0.3, 0.5),
            'pm_pct': np.random.uniform(0.1, 0.2),
            'founder_pct': np.random.uniform(0.05, 0.15)
        }
        # Normalize to sum to reasonable total
        total_role_pct = sum(role_dist.values())
        role_dist = {k: v / total_role_pct * 0.7 for k, v in role_dist.items()}  # 70% of audience
        
        # Engagement quality score
        avg_sentiment = (avg_sentiment_engineers + avg_sentiment_pms + avg_sentiment_founders) / 3
        engagement_quality = (
            (total_reactions_count / total_people) * 0.3 +
            (comment_volume_count / total_people) * 0.3 +
            max(0, avg_sentiment) * 0.4
        )
        
        post_data.append({
            # Post features
            'post_text': post_text,
            'post_topic': topic,
            'post_author_role': author_role,
            'post_length': post_length,
            'hashtags': has_hashtags,
            'mentions': has_mentions,
            'urls_present': has_urls,
            'time_posted': time_posted,
            
            # Aggregated Model A features (simulated)
            'pct_engineers_reacting': pct_engineers_reacting,
            'pct_pms_commenting': pct_pms_commenting,
            'pct_founders_engaging': pct_founders_engaging,
            'predicted_comment_volume': comment_volume,
            'reaction_like_pct': reaction_distribution.get('like', 0),
            'reaction_love_pct': reaction_distribution.get('love', 0),
            'reaction_insightful_pct': reaction_distribution.get('insightful', 0),
            'reaction_celebrate_pct': reaction_distribution.get('celebrate', 0),
            'reaction_none_pct': reaction_distribution.get('none', 0),
            'avg_sentiment_engineers': avg_sentiment_engineers,
            'avg_sentiment_pms': avg_sentiment_pms,
            'avg_sentiment_founders': avg_sentiment_founders,
            
            # Target variables
            'total_reactions': total_reactions_count,
            'comment_volume': comment_volume_count,
            'engineer_pct': role_dist['engineer_pct'],
            'pm_pct': role_dist['pm_pct'],
            'founder_pct': role_dist['founder_pct'],
            'engagement_quality': engagement_quality
        })
    
    df_post = pd.DataFrame(post_data)
    
    print(f"\\n✓ Generated {len(df_post)} posts with aggregated features")
    print(f"  Average reactions per post: {df_post['total_reactions'].mean():.1f}")
    print(f"  Average comments per post: {df_post['comment_volume'].mean():.1f}")
    print(f"  Average engagement quality: {df_post['engagement_quality'].mean():.3f}")
    
    # Save if path provided
    if save_path:
        save_dataset_to_file(df_post, save_path)
    
    return df_post

### Generate and Save Datasets

Use the functions above to generate synthetic datasets for Model A and Model B.

In [ ]:
# Configuration for dataset generation
DATASET_CONFIG = {
    'model_a': {
        'n_posts': 200,
        'interactions_per_post': 25,
        'save_path': 'synthetic_model_a_data.csv',  # Set to None to skip saving
        'seed': 42,
        'realism_level': 0.8
    },
    'model_b': {
        'n_posts': 200,
        'save_path': 'synthetic_model_b_data.csv',  # Set to None to skip saving
        'seed': 42,
        'realism_level': 0.8
    }
}

# Generate Model A dataset
print("=" * 70)
print("GENERATING MODEL A DATASET")
print("=" * 70)
df_model_a_synthetic = generate_synthetic_model_a_data(**DATASET_CONFIG['model_a'])

print("\\n" + "=" * 70)
print("GENERATING MODEL B DATASET")
print("=" * 70)
df_model_b_synthetic = generate_synthetic_model_b_data(**DATASET_CONFIG['model_b'])

print("\\n" + "=" * 70)
print("DATASET GENERATION COMPLETE")
print("=" * 70)
print(f"\\nModel A dataset: {len(df_model_a_synthetic)} rows, {len(df_model_a_synthetic.columns)} columns")
print(f"Model B dataset: {len(df_model_b_synthetic)} rows, {len(df_model_b_synthetic.columns)} columns")
print("\\nYou can now use these datasets to train Model A and Model B!")

## Data Loading and Simulation

In [ ]:
def load_or_simulate_data(data_path: Optional[str] = None, n_samples: int = 500) -> pd.DataFrame:
    """
    Load data from CSV/JSON file, directory of JSON files, or generate synthetic training data.
    
    Expected columns:
    - post_text: string
    - job_role: string (persona metadata)
    - affiliation: string (persona metadata)
    - account_age: int (days, persona metadata)
    - audience_size: int (context metadata)
    - baseline_engagement: float (context metadata)
    - time_window: string (context metadata)
    - pct_positive: float (target, 0-100)
    - pct_negative: float (target, 0-100)
    - comment_sentiment_dist: float (target, -1 to 1)
    - engagement_velocity: float (target, 0-1)
    """
    # Try to load existing data
    if data_path and os.path.exists(data_path):
        # Check if it's a directory (for loading multiple JSON files)
        if os.path.isdir(data_path):
            print(f"Loading data from directory: {data_path}")
            df = load_linkedin_json_data(data_path)
            print(f"✓ Loaded {len(df)} rows from {data_path}")
            return df
        # Otherwise, treat as a single file
        print(f"Loading data from {data_path}...")
        if data_path.endswith('.csv'):
            df = pd.read_csv(data_path)
        elif data_path.endswith('.json'):
            df = pd.read_json(data_path)
        else:
            raise ValueError(f"Unsupported file format: {data_path}")
        print(f"✓ Loaded {len(df)} rows from {data_path}")
        return df
    
    # Generate synthetic data
    print(f"Generating {n_samples} synthetic training examples...")
    
    # Post text templates
    post_templates = [
        "Excited to share my latest insights on {topic}! {opinion}",
        "Just finished reading an amazing article about {topic}. Here's what I learned: {insight}",
        "Hot take: {opinion} What do you think?",
        "After {experience}, I've realized that {insight}",
        "Breaking: {news} This changes everything!",
        "Quick thread on {topic}: 1/5 {point1}",
        "The future of {topic} is {prediction}. Here's why:",
        "I disagree with the common wisdom that {common_belief}. Instead, {alternative}",
        "5 lessons I learned from {experience}:",
        "Why {topic} matters more than you think: {reason}"
    ]
    
    topics = ["AI", "startups", "leadership", "technology", "productivity", "innovation", "data science", "marketing"]
    job_roles = ["Software Engineer", "Product Manager", "Data Scientist", "CEO", "CTO", "Marketing Director", "Designer", "Consultant"]
    affiliations = ["Tech Corp", "StartupXYZ", "BigTech Inc", "Consulting Group", "Agency", "Freelance", "Academia", "Government"]
    time_windows = ["morning", "afternoon", "evening", "weekend"]
    
    data = []
    for i in range(n_samples):
        # Generate post text
        template = np.random.choice(post_templates)
        topic = np.random.choice(topics)
        opinion = f"Game-changing innovation in {topic}"
        insight = f"{topic} is revolutionizing the industry"
        post_text = template.format(
            topic=topic,
            opinion=opinion,
            insight=insight,
            experience=f"working with {topic}",
            news=f"Major breakthrough in {topic}",
            point1=f"{topic} is evolving rapidly",
            prediction="bright",
            common_belief=f"{topic} is overhyped",
            alternative=f"{topic} is just getting started",
            reason=f"{topic} drives innovation"
        )
        
        # Persona metadata
        job_role = np.random.choice(job_roles)
        affiliation = np.random.choice(affiliations)
        account_age = np.random.randint(30, 3650)  # 1 month to 10 years in days
        
        # Context metadata
        audience_size = np.random.randint(100, 100000)
        baseline_engagement = np.random.uniform(0.01, 0.15)  # 1% to 15%
        time_window = np.random.choice(time_windows)
        
        # Simulate engagement metrics based on heuristics
        # Positive correlation: longer posts, certain topics, experienced accounts
        text_length_factor = len(post_text) / 200  # Normalize by average length
        account_credibility = min(account_age / 1000, 1.0)  # More credible with age
        topic_popularity = 0.7 if topic in ["AI", "technology", "startups"] else 0.5
        
        # Base engagement with some randomness
        base_positive = 20 + text_length_factor * 10 + account_credibility * 15 + topic_popularity * 10
        base_positive += np.random.normal(0, 10)
        pct_positive = np.clip(base_positive, 0, 100)
        
        base_negative = 5 + np.random.normal(0, 5)
        pct_negative = np.clip(base_negative, 0, 50)
        
        # Comment sentiment: positive posts get positive comments
        comment_sentiment_dist = (pct_positive - pct_negative) / 100
        comment_sentiment_dist = np.clip(comment_sentiment_dist, -1, 1)
        
        # Engagement velocity: how quickly engagement happens (early vs late)
        # High-engagement posts get early engagement
        engagement_velocity = min(pct_positive / 100, 1.0) * 0.7 + np.random.uniform(0, 0.3)
        engagement_velocity = np.clip(engagement_velocity, 0, 1)
        
        data.append({
            'post_text': post_text,
            'job_role': job_role,
            'affiliation': affiliation,
            'account_age': account_age,
            'audience_size': audience_size,
            'baseline_engagement': baseline_engagement,
            'time_window': time_window,
            'pct_positive': pct_positive,
            'pct_negative': pct_negative,
            'comment_sentiment_dist': comment_sentiment_dist,
            'engagement_velocity': engagement_velocity
        })
    
    df = pd.DataFrame(data)
    print(f"✓ Generated {len(df)} synthetic examples")
    return df

# Load or simulate data
df = load_or_simulate_data(DATA_PATH, n_samples=500)
print(f"\nDataset shape: {df.shape}")
print(f"\nFirst few rows:")
df.head()

## Data Cleaning and Preprocessing

In [ ]:
# Data cleaning and exploration
print("Dataset Info:")
print(df.info())
print("\nMissing values:")
print(df.isnull().sum())
print("\nTarget variable statistics:")
print(df[['pct_positive', 'pct_negative', 'comment_sentiment_dist', 'engagement_velocity']].describe())

# Handle missing values (if any)
df = df.dropna()

# Basic text normalization (optional - embeddings handle this well)
df['post_text_clean'] = df['post_text'].str.lower().str.strip()

print(f"\n✓ Cleaned dataset: {len(df)} rows")

## LLM Embedding Generation

Generate embeddings for post text using Google Gemini API

In [ ]:
def generate_embeddings(texts: List[str], model_name: str = EMBEDDING_MODEL, batch_size: int = 10) -> np.ndarray:
    """
    Generate embeddings for a list of texts using Google Gemini API.
    Includes batch processing, error handling, and fallback to simulated embeddings.
    """
    embeddings = []
    
    if not GEMINI_AVAILABLE or not GEMINI_API_KEY:
        print("⚠ Using simulated embeddings (random vectors)")
        # Generate random embeddings with same dimension as Gemini (768 for text-embedding-004, 768/1536/3072 for gemini-embedding-001)
        embedding_dim = 768
        for text in texts:
            # Create deterministic "embeddings" based on text hash for reproducibility
            np.random.seed(hash(text) % 2**32)
            emb = np.random.randn(embedding_dim)
            emb = emb / np.linalg.norm(emb)  # Normalize
            embeddings.append(emb)
        return np.array(embeddings)
    
    print(f"Generating embeddings for {len(texts)} texts using {model_name}...")
    
    # Try different API formats (support both old and new Google AI SDK)
    try:
        # Try new API format (google-genai package)
        try:
            from google import genai as genai_new
            client = genai_new.Client(api_key=GEMINI_API_KEY)
            use_new_api = True
        except:
            use_new_api = False
    except:
        use_new_api = False
    
    # Process in batches to handle rate limits
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        batch_embeddings = []
        
        for text in batch:
            try:
                if use_new_api:
                    # New API format
                    response = client.models.embed_content(
                        model=model_name,
                        contents=text
                    )
                    embedding = response.embeddings[0].values if hasattr(response.embeddings[0], 'values') else response.embeddings[0]
                else:
                    # Old API format (google-generativeai)
                    result = genai.embed_content(
                        model=model_name,
                        content=text,
                        task_type="RETRIEVAL_DOCUMENT"
                    )
                    # Handle different response formats
                    if isinstance(result, dict):
                        embedding = result.get('embedding', result.get('values', []))
                    else:
                        embedding = result.embedding if hasattr(result, 'embedding') else result
                
                batch_embeddings.append(embedding)
                
                # Small delay to avoid rate limits
                time.sleep(0.1)
            except Exception as e:
                print(f"Error embedding text {i}: {str(e)}")
                # Fallback: use random embedding
                embedding_dim = 768
                np.random.seed(hash(text) % 2**32)
                emb = np.random.randn(embedding_dim)
                emb = emb / np.linalg.norm(emb)
                batch_embeddings.append(emb)
        
        embeddings.extend(batch_embeddings)
        
        if (i + batch_size) % 50 == 0:
            print(f"  Processed {min(i + batch_size, len(texts))}/{len(texts)} texts...")
    
    print(f"✓ Generated {len(embeddings)} embeddings")
    return np.array(embeddings)

# Generate embeddings for all post texts
print("Generating text embeddings...")
text_embeddings = generate_embeddings(df['post_text_clean'].tolist())
print(f"Embedding shape: {text_embeddings.shape}")
print(f"Embedding dimension: {text_embeddings.shape[1]}")

## Feature Engineering

Combine text embeddings with persona and context metadata

In [ ]:
import glob
from datetime import datetime

def load_linkedin_json_data(data_folder: str) -> pd.DataFrame:
    """
    Load and transform LinkedIn post JSON files from the data folder.
    
    Args:
        data_folder: Path to folder containing JSON files
        
    Returns:
        DataFrame with transformed LinkedIn post data matching expected format
    """
    # Find all company post JSON files
    json_pattern = os.path.join(data_folder, '*company-posts*.json')
    json_files = glob.glob(json_pattern)
    
    if not json_files:
        raise ValueError(f"No company post JSON files found in {data_folder}")
    
    print(f"Found {len(json_files)} JSON file(s) to load...")
    
    all_posts = []
    
    for json_file in json_files:
        print(f"Loading {os.path.basename(json_file)}...")
        with open(json_file, 'r', encoding='utf-8') as f:
            posts = json.load(f)
        
        if not isinstance(posts, list):
            posts = [posts]
        
        print(f"  Found {len(posts)} posts")
        all_posts.extend(posts)
    
    print(f"Total posts loaded: {len(all_posts)}")
    print("Transforming data...")
    
    transformed_data = []
    
    for post in all_posts:
        # Skip posts without required fields
        if not post.get('text') or not post.get('stats'):
            continue
        
        # Extract basic fields
        post_text = post.get('text', '')
        stats = post.get('stats', {})
        author = post.get('author', {})
        posted_at = post.get('posted_at', {})
        
        # Calculate engagement metrics
        total_reactions = stats.get('total_reactions', 0)
        like_count = stats.get('like', 0)
        love_count = stats.get('love', 0)
        celebrate_count = stats.get('celebrate', 0)
        support_count = stats.get('support', 0)
        insight_count = stats.get('insight', 0)
        comments_count = stats.get('comments', 0)
        reposts_count = stats.get('reposts', 0)
        
        # Calculate pct_positive (positive reactions / total reactions * 100)
        positive_reactions = like_count + love_count + celebrate_count + support_count
        if total_reactions > 0:
            pct_positive = (positive_reactions / total_reactions) * 100
            pct_negative = (insight_count / total_reactions) * 100
        else:
            pct_positive = 0.0
            pct_negative = 0.0
        
        # Audience size from follower count
        audience_size = author.get('follower_count', 0)
        if audience_size == 0:
            audience_size = 1000  # Default minimum
        
        # Calculate baseline engagement
        total_engagement = total_reactions + comments_count + reposts_count
        baseline_engagement = total_engagement / max(audience_size, 1)
        
        # Calculate comment sentiment distribution (normalized -1 to 1)
        if total_reactions > 0:
            comment_sentiment_dist = (pct_positive - pct_negative) / 100
            comment_sentiment_dist = np.clip(comment_sentiment_dist, -1, 1)
        else:
            comment_sentiment_dist = 0.0
        
        # Extract time window from timestamp
        timestamp = posted_at.get('timestamp', 0)
        if timestamp > 0:
            # Convert timestamp (milliseconds) to datetime
            post_datetime = datetime.fromtimestamp(timestamp / 1000)
            hour = post_datetime.hour
            weekday = post_datetime.weekday()  # 0 = Monday, 6 = Sunday
            
            if weekday >= 5:  # Saturday or Sunday
                time_window = 'weekend'
            elif hour < 12:
                time_window = 'morning'
            elif hour < 17:
                time_window = 'afternoon'
            else:
                time_window = 'evening'
        else:
            time_window = 'afternoon'  # Default
        
        # Extract affiliation from source_company
        affiliation = post.get('source_company', 'Unknown')
        if affiliation and isinstance(affiliation, str):
            affiliation = affiliation.capitalize()
        else:
            affiliation = 'Unknown'
        
        # Infer job_role from company name or use default
        # For company posts, we'll use a default role based on the company
        company_name = affiliation.lower()
        if 'google' in company_name or 'microsoft' in company_name or 'apple' in company_name:
            job_role = 'Software Engineer'
        elif 'meta' in company_name or 'facebook' in company_name:
            job_role = 'Product Manager'
        else:
            job_role = 'Product Manager'  # Default for company posts
        
        # Account age - use default since we don't have account creation date
        # For company accounts, use a reasonable default (e.g., 5 years)
        account_age = 1825  # 5 years in days
        
        # Calculate engagement velocity
        # Estimate based on post age and engagement rate
        # Newer posts with high engagement = high velocity
        if timestamp > 0:
            # Calculate post age in days
            current_time = datetime.now().timestamp() * 1000
            post_age_days = (current_time - timestamp) / (1000 * 60 * 60 * 24)
            
            # High engagement rate + recent post = high velocity
            engagement_rate = baseline_engagement
            if post_age_days < 1:  # Less than 1 day old
                velocity_factor = 0.9
            elif post_age_days < 7:  # Less than 1 week old
                velocity_factor = 0.7
            else:
                velocity_factor = 0.5
            
            engagement_velocity = min(engagement_rate * 10 * velocity_factor, 1.0)
            engagement_velocity = max(engagement_velocity, 0.0)
        else:
            engagement_velocity = 0.5  # Default
        
        transformed_data.append({
            'post_text': post_text,
            'job_role': job_role,
            'affiliation': affiliation,
            'account_age': account_age,
            'audience_size': audience_size,
            'baseline_engagement': baseline_engagement,
            'time_window': time_window,
            'pct_positive': pct_positive,
            'pct_negative': pct_negative,
            'comment_sentiment_dist': comment_sentiment_dist,
            'engagement_velocity': engagement_velocity
        })
    
    df = pd.DataFrame(transformed_data)
    print(f"✓ Transformed {len(df)} posts")
    return df

In [ ]:
def load_linkedin_json_data(data_folder: str) -> pd.DataFrame:
    """
    Load and transform LinkedIn post JSON files from the data folder.
    
    Args:
        data_folder: Path to folder containing JSON files
        
    Returns:
        DataFrame with transformed LinkedIn post data matching expected format
    """
    import glob
    from datetime import datetime
    
    # Find all company post JSON files
    json_pattern = os.path.join(data_folder, '*company-posts*.json')
    json_files = glob.glob(json_pattern)
    
    if not json_files:
        raise ValueError(f"No company post JSON files found in {data_folder}")
    
    print(f"Found {len(json_files)} JSON file(s) to load...")
    
    all_posts = []
    
    for json_file in json_files:
        print(f"Loading {os.path.basename(json_file)}...")
        with open(json_file, 'r', encoding='utf-8') as f:
            posts = json.load(f)
        
        if not isinstance(posts, list):
            posts = [posts]
        
        print(f"  Found {len(posts)} posts")
        all_posts.extend(posts)
    
    print(f"Total posts loaded: {len(all_posts)}")
    print("Transforming data...")
    
    transformed_data = []
    
    for post in all_posts:
        # Skip posts without required fields
        if not post.get('text') or not post.get('stats'):
            continue
        
        # Extract basic fields
        post_text = post.get('text', '')
        stats = post.get('stats', {})
        author = post.get('author', {})
        posted_at = post.get('posted_at', {})
        
        # Calculate engagement metrics
        total_reactions = stats.get('total_reactions', 0)
        like_count = stats.get('like', 0)
        love_count = stats.get('love', 0)
        celebrate_count = stats.get('celebrate', 0)
        support_count = stats.get('support', 0)
        insight_count = stats.get('insight', 0)
        comments_count = stats.get('comments', 0)
        reposts_count = stats.get('reposts', 0)
        
        # Calculate pct_positive (positive reactions / total reactions * 100)
        positive_reactions = like_count + love_count + celebrate_count + support_count
        if total_reactions > 0:
            pct_positive = (positive_reactions / total_reactions) * 100
            pct_negative = (insight_count / total_reactions) * 100
        else:
            pct_positive = 0.0
            pct_negative = 0.0
        
        # Audience size from follower count
        audience_size = author.get('follower_count', 0)
        if audience_size == 0:
            audience_size = 1000  # Default minimum
        
        # Calculate baseline engagement
        total_engagement = total_reactions + comments_count + reposts_count
        baseline_engagement = total_engagement / max(audience_size, 1)
        
        # Calculate comment sentiment distribution (normalized -1 to 1)
        if total_reactions > 0:
            comment_sentiment_dist = (pct_positive - pct_negative) / 100
            comment_sentiment_dist = np.clip(comment_sentiment_dist, -1, 1)
        else:
            comment_sentiment_dist = 0.0
        
        # Extract time window from timestamp
        timestamp = posted_at.get('timestamp', 0)
        if timestamp > 0:
            # Convert timestamp (milliseconds) to datetime
            post_datetime = datetime.fromtimestamp(timestamp / 1000)
            hour = post_datetime.hour
            weekday = post_datetime.weekday()  # 0 = Monday, 6 = Sunday
            
            if weekday >= 5:  # Saturday or Sunday
                time_window = 'weekend'
            elif hour < 12:
                time_window = 'morning'
            elif hour < 17:
                time_window = 'afternoon'
            else:
                time_window = 'evening'
        else:
            time_window = 'afternoon'  # Default
        
        # Extract affiliation from source_company
        affiliation = post.get('source_company', 'Unknown')
        if affiliation and isinstance(affiliation, str):
            affiliation = affiliation.capitalize()
        else:
            affiliation = 'Unknown'
        
        # Infer job_role from company name or use default
        # For company posts, we'll use a default role based on the company
        company_name = affiliation.lower()
        if 'google' in company_name or 'microsoft' in company_name or 'apple' in company_name:
            job_role = 'Software Engineer'
        elif 'meta' in company_name or 'facebook' in company_name:
            job_role = 'Product Manager'
        else:
            job_role = 'Product Manager'  # Default for company posts
        
        # Account age - use default since we don't have account creation date
        # For company accounts, use a reasonable default (e.g., 5 years)
        account_age = 1825  # 5 years in days
        
        # Calculate engagement velocity
        # Estimate based on post age and engagement rate
        # Newer posts with high engagement = high velocity
        if timestamp > 0:
            # Calculate post age in days
            current_time = datetime.now().timestamp() * 1000
            post_age_days = (current_time - timestamp) / (1000 * 60 * 60 * 24)
            
            # High engagement rate + recent post = high velocity
            engagement_rate = baseline_engagement
            if post_age_days < 1:  # Less than 1 day old
                velocity_factor = 0.9
            elif post_age_days < 7:  # Less than 1 week old
                velocity_factor = 0.7
            else:
                velocity_factor = 0.5
            
            engagement_velocity = min(engagement_rate * 10 * velocity_factor, 1.0)
            engagement_velocity = max(engagement_velocity, 0.0)
        else:
            engagement_velocity = 0.5  # Default
        
        transformed_data.append({
            'post_text': post_text,
            'job_role': job_role,
            'affiliation': affiliation,
            'account_age': account_age,
            'audience_size': audience_size,
            'baseline_engagement': baseline_engagement,
            'time_window': time_window,
            'pct_positive': pct_positive,
            'pct_negative': pct_negative,
            'comment_sentiment_dist': comment_sentiment_dist,
            'engagement_velocity': engagement_velocity
        })
    
    df = pd.DataFrame(transformed_data)
    print(f"✓ Transformed {len(df)} posts")
    return df

In [ ]:
def prepare_features(df: pd.DataFrame, text_embeddings: np.ndarray) -> Tuple[np.ndarray, np.ndarray, Dict]:
    """
    Prepare feature matrix by combining embeddings with metadata.
    Returns: (X, y, feature_info)
    """
    # Encode categorical features
    le_job = LabelEncoder()
    le_affiliation = LabelEncoder()
    le_time = LabelEncoder()
    
    job_encoded = le_job.fit_transform(df['job_role'])
    affiliation_encoded = le_affiliation.fit_transform(df['affiliation'])
    time_encoded = le_time.fit_transform(df['time_window'])
    
    # One-hot encode categorical features for better representation
    ohe = OneHotEncoder(sparse_output=False, drop='first')
    categorical_features = ohe.fit_transform(
        df[['job_role', 'affiliation', 'time_window']]
    )
    
    # Normalize numerical features
    scaler = StandardScaler()
    numerical_features = scaler.fit_transform(
        df[['account_age', 'audience_size', 'baseline_engagement']]
    )
    
    # Combine all features: embeddings + categorical + numerical
    X = np.hstack([
        text_embeddings,  # Text embeddings (e.g., 768 dims)
        categorical_features,  # One-hot encoded categoricals
        numerical_features  # Normalized numerical features
    ])
    
    # Extract target variables
    y = df[['pct_positive', 'pct_negative', 'comment_sentiment_dist', 'engagement_velocity']].values
    
    feature_info = {
        'embedding_dim': text_embeddings.shape[1],
        'categorical_dim': categorical_features.shape[1],
        'numerical_dim': numerical_features.shape[1],
        'total_dim': X.shape[1],
        'label_encoders': {
            'job_role': le_job,
            'affiliation': le_affiliation,
            'time_window': le_time
        },
        'one_hot_encoder': ohe,
        'scaler': scaler
    }
    
    return X, y, feature_info

# Prepare features
X, y, feature_info = prepare_features(df, text_embeddings)

print(f"Feature matrix shape: {X.shape}")
print(f"Target matrix shape: {y.shape}")
print(f"\nFeature breakdown:")
print(f"  - Text embeddings: {feature_info['embedding_dim']} dimensions")
print(f"  - Categorical features: {feature_info['categorical_dim']} dimensions")
print(f"  - Numerical features: {feature_info['numerical_dim']} dimensions")
print(f"  - Total features: {feature_info['total_dim']} dimensions")
print(f"\nTarget variables: pct_positive, pct_negative, comment_sentiment_dist, engagement_velocity")

## Model Training

Train XGBoost multi-output regression model

In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

# Create XGBoost model with MultiOutputRegressor wrapper
# This trains one XGBoost model per target variable
base_model = XGBRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

model = MultiOutputRegressor(base_model)

print("\nTraining model...")
model.fit(X_train, y_train)
print("✓ Model training complete")

# Make predictions
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

print(f"\nPredictions shape: {y_test_pred.shape}")

## Model Evaluation

Calculate metrics and analyze performance

In [ ]:
def evaluate_model(y_true: np.ndarray, y_pred: np.ndarray, target_names: List[str]) -> pd.DataFrame:
    """
    Calculate evaluation metrics for each target variable.
    """
    results = []
    
    for i, target_name in enumerate(target_names):
        y_true_i = y_true[:, i]
        y_pred_i = y_pred[:, i]
        
        mae = mean_absolute_error(y_true_i, y_pred_i)
        rmse = np.sqrt(mean_squared_error(y_true_i, y_pred_i))
        r2 = r2_score(y_true_i, y_pred_i)
        
        # Calculate correlation
        correlation = np.corrcoef(y_true_i, y_pred_i)[0, 1]
        
        results.append({
            'target': target_name,
            'MAE': mae,
            'RMSE': rmse,
            'R²': r2,
            'Correlation': correlation
        })
    
    return pd.DataFrame(results)

# Evaluate on train and test sets
target_names = ['pct_positive', 'pct_negative', 'comment_sentiment_dist', 'engagement_velocity']

train_metrics = evaluate_model(y_train, y_train_pred, target_names)
test_metrics = evaluate_model(y_test, y_test_pred, target_names)

print("Training Set Metrics:")
print(train_metrics.to_string(index=False))
print("\nTest Set Metrics:")
print(test_metrics.to_string(index=False))

## Uncertainty Quantification

Estimate prediction uncertainty and identify failure modes

In [ ]:
def calculate_uncertainty(y_true: np.ndarray, y_pred: np.ndarray, target_names: List[str]) -> pd.DataFrame:
    """
    Calculate prediction errors and uncertainty metrics.
    """
    uncertainty_data = []
    
    for i, target_name in enumerate(target_names):
        y_true_i = y_true[:, i]
        y_pred_i = y_pred[:, i]
        
        # Absolute error
        abs_error = np.abs(y_true_i - y_pred_i)
        
        # Relative error (percentage)
        # Avoid division by zero
        with np.errstate(divide='ignore', invalid='ignore'):
            rel_error = np.where(
                y_true_i != 0,
                abs_error / np.abs(y_true_i) * 100,
                abs_error
            )
        
        uncertainty_data.append({
            'target': target_name,
            'mean_abs_error': np.mean(abs_error),
            'std_abs_error': np.std(abs_error),
            'max_abs_error': np.max(abs_error),
            'mean_rel_error_pct': np.mean(rel_error[rel_error != np.inf]),
            'high_uncertainty_threshold': np.percentile(abs_error, 90)  # 90th percentile
        })
    
    return pd.DataFrame(uncertainty_data)

# Calculate uncertainty metrics
test_uncertainty = calculate_uncertainty(y_test, y_test_pred, target_names)
print("Uncertainty Metrics (Test Set):")
print(test_uncertainty.to_string(index=False))

# Identify high-uncertainty predictions (potential failure modes)
print("\nIdentifying high-uncertainty predictions...")
abs_errors = np.abs(y_test - y_test_pred)
high_uncertainty_thresholds = test_uncertainty['high_uncertainty_threshold'].values

high_uncertainty_mask = np.any(
    abs_errors > high_uncertainty_thresholds.reshape(1, -1),
    axis=1
)

print(f"High-uncertainty predictions: {np.sum(high_uncertainty_mask)} / {len(y_test)} ({np.sum(high_uncertainty_mask)/len(y_test)*100:.1f}%)")
print("\nThese are potential failure modes where the model is less confident.")

## Feature Importance

Analyze which features contribute most to predictions

In [ ]:
# Extract feature importance from XGBoost models
# Since we use MultiOutputRegressor, we have one model per target
feature_importance_data = []

for i, target_name in enumerate(target_names):
    # Get the underlying XGBoost model for this target
    xgb_model = model.estimators_[i]
    importances = xgb_model.feature_importances_
    
    # Get top 20 most important features
    top_indices = np.argsort(importances)[-20:][::-1]
    top_importances = importances[top_indices]
    
    # Categorize features
    feature_categories = []
    for idx in top_indices:
        if idx < feature_info['embedding_dim']:
            feature_categories.append(f'Embedding_{idx}')
        elif idx < feature_info['embedding_dim'] + feature_info['categorical_dim']:
            feature_categories.append(f'Categorical_{idx - feature_info["embedding_dim"]}')
        else:
            feature_categories.append(f'Numerical_{idx - feature_info["embedding_dim"] - feature_info["categorical_dim"]}')
    
    feature_importance_data.append({
        'target': target_name,
        'top_features': feature_categories,
        'top_importances': top_importances
    })

# Display feature importance summary
print("Top 10 Most Important Features per Target:")
for data in feature_importance_data:
    print(f"\n{data['target']}:")
    for feat, imp in zip(data['top_features'][:10], data['top_importances'][:10]):
        print(f"  {feat}: {imp:.4f}")

## Visualizations

Create plots to visualize model performance and results

In [ ]:
# 1. Prediction vs Actual scatter plots
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.flatten()

for i, target_name in enumerate(target_names):
    ax = axes[i]
    y_true_i = y_test[:, i]
    y_pred_i = y_test_pred[:, i]
    
    # Scatter plot
    ax.scatter(y_true_i, y_pred_i, alpha=0.6, s=50)
    
    # Perfect prediction line
    min_val = min(y_true_i.min(), y_pred_i.min())
    max_val = max(y_true_i.max(), y_pred_i.max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect Prediction')
    
    # Calculate R² for this target
    r2 = r2_score(y_true_i, y_pred_i)
    ax.set_xlabel(f'Actual {target_name}')
    ax.set_ylabel(f'Predicted {target_name}')
    ax.set_title(f'{target_name}\nR² = {r2:.3f}')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 2. Residual plots
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.flatten()

for i, target_name in enumerate(target_names):
    ax = axes[i]
    y_true_i = y_test[:, i]
    y_pred_i = y_test_pred[:, i]
    residuals = y_true_i - y_pred_i
    
    ax.scatter(y_pred_i, residuals, alpha=0.6, s=50)
    ax.axhline(y=0, color='r', linestyle='--', lw=2)
    ax.set_xlabel(f'Predicted {target_name}')
    ax.set_ylabel('Residuals (Actual - Predicted)')
    ax.set_title(f'Residual Plot: {target_name}')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 3. Metrics comparison bar chart
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(target_names))
width = 0.35

# Plot R² scores
train_r2 = train_metrics['R²'].values
test_r2 = test_metrics['R²'].values

ax.bar(x - width/2, train_r2, width, label='Train R²', alpha=0.8)
ax.bar(x + width/2, test_r2, width, label='Test R²', alpha=0.8)

ax.set_xlabel('Target Variable')
ax.set_ylabel('R² Score')
ax.set_title('Model Performance: R² Scores')
ax.set_xticks(x)
ax.set_xticklabels(target_names, rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# 4. Error distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.flatten()

for i, target_name in enumerate(target_names):
    ax = axes[i]
    y_true_i = y_test[:, i]
    y_pred_i = y_test_pred[:, i]
    abs_errors = np.abs(y_true_i - y_pred_i)
    
    ax.hist(abs_errors, bins=30, alpha=0.7, edgecolor='black')
    ax.axvline(np.mean(abs_errors), color='r', linestyle='--', lw=2, label=f'Mean: {np.mean(abs_errors):.2f}')
    ax.set_xlabel('Absolute Error')
    ax.set_ylabel('Frequency')
    ax.set_title(f'Error Distribution: {target_name}')
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# 5. Correlation heatmap of predictions
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Actual values correlation
actual_df = pd.DataFrame(y_test, columns=target_names)
corr_actual = actual_df.corr()
sns.heatmap(corr_actual, annot=True, fmt='.2f', cmap='coolwarm', center=0, 
            square=True, ax=ax1, cbar_kws={'label': 'Correlation'})
ax1.set_title('Correlation: Actual Values')

# Predicted values correlation
pred_df = pd.DataFrame(y_test_pred, columns=target_names)
corr_pred = pred_df.corr()
sns.heatmap(corr_pred, annot=True, fmt='.2f', cmap='coolwarm', center=0, 
            square=True, ax=ax2, cbar_kws={'label': 'Correlation'})
ax2.set_title('Correlation: Predicted Values')

plt.tight_layout()
plt.show()

In [ ]:
# 6. Feature importance visualization (for first target as example)
target_idx = 0
xgb_model = model.estimators_[target_idx]
importances = xgb_model.feature_importances_

# Get top 15 features
top_n = 15
top_indices = np.argsort(importances)[-top_n:][::-1]
top_importances = importances[top_indices]

# Create feature labels
feature_labels = []
for idx in top_indices:
    if idx < feature_info['embedding_dim']:
        feature_labels.append(f'Embedding[{idx}]')
    elif idx < feature_info['embedding_dim'] + feature_info['categorical_dim']:
        feature_labels.append(f'Categorical[{idx - feature_info["embedding_dim"]}]')
    else:
        num_idx = idx - feature_info['embedding_dim'] - feature_info['categorical_dim']
        num_features = ['account_age', 'audience_size', 'baseline_engagement']
        feature_labels.append(num_features[num_idx] if num_idx < len(num_features) else f'Num[{num_idx}]')

fig, ax = plt.subplots(figsize=(10, 8))
ax.barh(range(len(top_importances)), top_importances)
ax.set_yticks(range(len(top_importances)))
ax.set_yticklabels(feature_labels)
ax.set_xlabel('Feature Importance')
ax.set_title(f'Top {top_n} Features: {target_names[target_idx]}')
ax.invert_yaxis()
plt.tight_layout()
plt.show()

## Two-Model Architecture: Individual & Post-Level Prediction

This section implements a two-model pipeline that provides more interpretable and granular predictions:

**Model A (Individual-Level)**: Predicts how specific role archetypes react to posts
- One row = one person × one post interaction
- Predicts: reaction type, comment presence, comment sentiment, comment intent
- Focuses on role archetypes (Software Engineer, PM, Founder, etc.), not individuals

**Model B (Post-Level)**: Aggregates Model A predictions to forecast overall post performance
- One row = one post
- Uses aggregated Model A predictions as features
- Predicts: total reactions, comment volume, role distribution, engagement quality

**Pipeline Flow**: Post → Model A (role predictions) → Aggregate → Model B → Post-level metrics

### Model A: Individual-Level Reaction & Comment Model

Model A predicts how different role archetypes will react to a post.

In [ ]:
def load_individual_level_data(
    post_level_df: pd.DataFrame,
    interactions_per_post: int = 20,
    individual_data_path: Optional[str] = None
) -> pd.DataFrame:
    """
    Load or simulate individual-level interaction data (person × post).
    
    Expected columns if loading from file:
    - Post features: post_text, post_topic, post_author_role, post_length, hashtags, mentions, urls_present, time_posted
    - Person features: commenter_role, commenter_industry, commenter_seniority
    - Labels: reaction_type, commented, comment_text, comment_sentiment, comment_intent
    
    If data not available, simulates individual interactions from post-level data.
    """
    # Try to load existing individual-level data
    if individual_data_path and os.path.exists(individual_data_path):
        print(f"Loading individual-level data from {individual_data_path}...")
        if individual_data_path.endswith('.csv'):
            df_individual = pd.read_csv(individual_data_path)
        elif individual_data_path.endswith('.json'):
            df_individual = pd.read_json(individual_data_path)
        else:
            raise ValueError(f"Unsupported file format: {individual_data_path}")
        print(f"✓ Loaded {len(df_individual)} individual interactions")
        return df_individual
    
    # Simulate individual-level data from post-level data
    print(f"Simulating individual-level interactions ({interactions_per_post} per post)...")
    
    # Role archetypes with normalized buckets
    role_archetypes = {
        'Software Engineer': {'industry': 'Tech', 'seniority': 'Mid'},
        'Product Manager': {'industry': 'Tech', 'seniority': 'Mid'},
        'Data Scientist': {'industry': 'Tech', 'seniority': 'Mid'},
        'CEO': {'industry': 'Tech', 'seniority': 'Exec'},
        'CTO': {'industry': 'Tech', 'seniority': 'Exec'},
        'Founder': {'industry': 'Tech', 'seniority': 'Exec'},
        'Recruiter': {'industry': 'Tech', 'seniority': 'Mid'},
        'Marketing Director': {'industry': 'Tech', 'seniority': 'Senior'},
        'Designer': {'industry': 'Tech', 'seniority': 'Mid'},
        'Consultant': {'industry': 'Consulting', 'seniority': 'Senior'},
        'Student': {'industry': 'Education', 'seniority': 'Junior'}
    }
    
    industries = ['Tech', 'Healthcare', 'Finance', 'Education', 'Consulting']
    seniorities = ['Junior', 'Mid', 'Senior', 'Exec']
    reaction_types = ['like', 'love', 'insightful', 'celebrate', 'none']
    comment_intents = ['praise', 'agreement', 'question', 'criticism', 'insight_addition', 'none']
    
    individual_data = []
    
    for idx, post_row in post_level_df.iterrows():
        post_text = post_row['post_text']
        post_topic = 'AI' if 'AI' in post_text or 'artificial intelligence' in post_text.lower() else \
                     'startups' if 'startup' in post_text.lower() else \
                     'technology' if 'tech' in post_text.lower() else 'general'
        
        # Sample role archetypes for this post
        sampled_roles = np.random.choice(list(role_archetypes.keys()), 
                                        size=min(interactions_per_post, len(role_archetypes)),
                                        replace=False)
        
        for role in sampled_roles:
            role_info = role_archetypes[role]
            
            # Determine reaction based on role-post alignment
            # Engineers more likely to react to tech posts, founders to startup posts, etc.
            base_reaction_prob = 0.7
            if (role in ['Software Engineer', 'Data Scientist', 'CTO'] and post_topic in ['AI', 'technology']) or \
               (role == 'Founder' and post_topic == 'startups') or \
               (role == 'CEO' and post_topic in ['startups', 'technology']):
                base_reaction_prob = 0.85
            
            # Reaction type
            if np.random.random() < base_reaction_prob:
                reaction_type = np.random.choice(['like', 'love', 'insightful', 'celebrate'], 
                                                p=[0.5, 0.2, 0.2, 0.1])
            else:
                reaction_type = 'none'
            
            # Comment probability (lower than reaction)
            comment_prob = 0.15 if reaction_type != 'none' else 0.02
            commented = np.random.random() < comment_prob
            
            # Comment sentiment (if commented)
            if commented:
                # Sentiment based on reaction type
                if reaction_type in ['love', 'celebrate']:
                    comment_sentiment = np.random.uniform(0.6, 1.0)
                    comment_intent = np.random.choice(['praise', 'agreement'], p=[0.6, 0.4])
                elif reaction_type == 'insightful':
                    comment_sentiment = np.random.uniform(0.3, 0.8)
                    comment_intent = np.random.choice(['insight_addition', 'agreement', 'question'], p=[0.5, 0.3, 0.2])
                elif reaction_type == 'like':
                    comment_sentiment = np.random.uniform(0.0, 0.7)
                    comment_intent = np.random.choice(['agreement', 'question', 'praise'], p=[0.4, 0.3, 0.3])
                else:
                    comment_sentiment = np.random.uniform(-0.5, 0.5)
                    comment_intent = np.random.choice(['criticism', 'question'], p=[0.4, 0.6])
                
                comment_text = f"Sample comment from {role}"
            else:
                comment_sentiment = 0.0
                comment_intent = 'none'
                comment_text = ''
            
            # Post metadata
            post_length = len(post_text.split())
            has_hashtags = '#' in post_text
            has_mentions = '@' in post_text
            has_urls = 'http' in post_text.lower()
            
            individual_data.append({
                # Post features
                'post_text': post_text,
                'post_topic': post_topic,
                'post_author_role': post_row.get('job_role', 'CEO'),
                'post_length': post_length,
                'hashtags': has_hashtags,
                'mentions': has_mentions,
                'urls_present': has_urls,
                'time_posted': post_row.get('time_window', 'morning'),
                
                # Person features
                'commenter_role': role,
                'commenter_industry': role_info['industry'],
                'commenter_seniority': role_info['seniority'],
                
                # Labels
                'reaction_type': reaction_type,
                'commented': commented,
                'comment_text': comment_text,
                'comment_sentiment': comment_sentiment,
                'comment_intent': comment_intent
            })
    
    df_individual = pd.DataFrame(individual_data)
    print(f"✓ Generated {len(df_individual)} individual interactions from {len(post_level_df)} posts")
    return df_individual

# Load individual-level data
INDIVIDUAL_DATA_PATH = 'individual_interactions.csv'  # Set to None to simulate
df_individual = load_individual_level_data(df, interactions_per_post=15)
print(f"\\nIndividual-level dataset shape: {df_individual.shape}")
print(f"\\nFirst few rows:")
df_individual.head()

In [ ]:
def prepare_model_a_features(df_individual: pd.DataFrame, text_embeddings_dict: Optional[Dict] = None) -> Tuple[np.ndarray, Dict, Dict]:
    """
    Prepare features for Model A (individual-level predictions).
    
    Features:
    - Post text embeddings
    - Post metadata (topic, author role, length, hashtags, mentions, URLs, time)
    - Person features (role, industry, seniority)
    
    Returns:
    - X: Feature matrix
    - y_dict: Dictionary of target variables
    - feature_info_a: Feature information (encoders, scalers)
    """
    print("Preparing Model A features...")
    
    # Generate embeddings for unique posts (cache to avoid recomputation)
    if text_embeddings_dict is None:
        unique_posts = df_individual[['post_text']].drop_duplicates()
        unique_posts['post_text_clean'] = unique_posts['post_text'].str.lower().str.strip()
        print(f"Generating embeddings for {len(unique_posts)} unique posts...")
        post_embeddings = generate_embeddings(unique_posts['post_text_clean'].tolist())
        text_embeddings_dict = dict(zip(unique_posts['post_text'], post_embeddings))
    
    # Map embeddings to each row
    df_individual['post_text_clean'] = df_individual['post_text'].str.lower().str.strip()
    post_embeddings_list = [text_embeddings_dict[post] for post in df_individual['post_text']]
    post_embeddings_array = np.array(post_embeddings_list)
    
    # Encode post metadata
    le_post_topic = LabelEncoder()
    le_post_author = LabelEncoder()
    le_time = LabelEncoder()
    
    post_topic_encoded = le_post_topic.fit_transform(df_individual['post_topic'])
    post_author_encoded = le_post_author.fit_transform(df_individual['post_author_role'])
    time_encoded = le_time.fit_transform(df_individual['time_posted'])
    
    # One-hot encode post categoricals
    ohe_post = OneHotEncoder(sparse_output=False, drop='first')
    post_categorical = ohe_post.fit_transform(
        df_individual[['post_topic', 'post_author_role', 'time_posted']]
    )
    
    # Post numerical features
    post_numerical = df_individual[['post_length', 'hashtags', 'mentions', 'urls_present']].astype(float).values
    scaler_post_numerical = StandardScaler()
    post_numerical_scaled = scaler_post_numerical.fit_transform(post_numerical)
    
    # Encode person features
    le_role = LabelEncoder()
    le_industry = LabelEncoder()
    le_seniority = LabelEncoder()
    
    role_encoded = le_role.fit_transform(df_individual['commenter_role'])
    industry_encoded = le_industry.fit_transform(df_individual['commenter_industry'])
    seniority_encoded = le_seniority.fit_transform(df_individual['commenter_seniority'])
    
    # One-hot encode person categoricals
    ohe_person = OneHotEncoder(sparse_output=False, drop='first')
    person_categorical = ohe_person.fit_transform(
        df_individual[['commenter_role', 'commenter_industry', 'commenter_seniority']]
    )
    
    # Combine all features
    X = np.hstack([
        post_embeddings_array,  # Post text embeddings
        post_categorical,  # Post categorical features
        post_numerical_scaled,  # Post numerical features
        person_categorical  # Person features
    ])
    
    # Prepare target variables
    y_dict = {
        'reaction_type': df_individual['reaction_type'].values,
        'commented': df_individual['commented'].astype(int).values,
        'comment_sentiment': df_individual['comment_sentiment'].values,
        'comment_intent': df_individual['comment_intent'].values
    }
    
    feature_info_a = {
        'embedding_dim': post_embeddings_array.shape[1],
        'post_categorical_dim': post_categorical.shape[1],
        'post_numerical_dim': post_numerical_scaled.shape[1],
        'person_categorical_dim': person_categorical.shape[1],
        'total_dim': X.shape[1],
        'text_embeddings_dict': text_embeddings_dict,
        'encoders': {
            'post_topic': le_post_topic,
            'post_author': le_post_author,
            'time': le_time,
            'role': le_role,
            'industry': le_industry,
            'seniority': le_seniority
        },
        'one_hot_encoders': {
            'post': ohe_post,
            'person': ohe_person
        },
        'scaler_post_numerical': scaler_post_numerical
    }
    
    print(f"✓ Feature matrix shape: {X.shape}")
    print(f"  - Post embeddings: {post_embeddings_array.shape[1]} dims")
    print(f"  - Post categorical: {post_categorical.shape[1]} dims")
    print(f"  - Post numerical: {post_numerical_scaled.shape[1]} dims")
    print(f"  - Person categorical: {person_categorical.shape[1]} dims")
    
    return X, y_dict, feature_info_a

# Prepare Model A features
X_a, y_dict_a, feature_info_a = prepare_model_a_features(df_individual)
print(f"\\nTarget variables:")
for key, values in y_dict_a.items():
    if key == 'comment_sentiment':
        print(f"  {key}: {len(values)} values, range [{values.min():.2f}, {values.max():.2f}]")
    else:
        unique_vals = np.unique(values)
        print(f"  {key}: {len(unique_vals)} unique values - {list(unique_vals)[:5]}{'...' if len(unique_vals) > 5 else ''}")

In [ ]:
# Split data for Model A
X_a_train, X_a_test, y_a_train, y_a_test = train_test_split(
    X_a, y_dict_a, test_size=0.2, random_state=42
)

print(f"Model A Training set: {X_a_train.shape[0]} samples")
print(f"Model A Test set: {X_a_test.shape[0]} samples")

# Train Model A components
print("\\nTraining Model A components...")

# 1. Reaction type classifier (multi-class)
print("  1. Training reaction type classifier...")
model_a_reaction = XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42,
    n_jobs=-1,
    eval_metric='mlogloss'
)
model_a_reaction.fit(X_a_train, y_a_train['reaction_type'])
print("     ✓ Reaction type classifier trained")

# 2. Comment presence classifier (binary)
print("  2. Training comment presence classifier...")
model_a_comment = XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42,
    n_jobs=-1,
    eval_metric='logloss'
)
model_a_comment.fit(X_a_train, y_a_train['commented'])
print("     ✓ Comment presence classifier trained")

# 3. Comment sentiment regressor (only for commented=True)
print("  3. Training comment sentiment regressor...")
commented_mask_train = y_a_train['commented'] == 1
commented_mask_test = y_a_test['commented'] == 1

if commented_mask_train.sum() > 0:
    model_a_sentiment = XGBRegressor(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        random_state=42,
        n_jobs=-1
    )
    model_a_sentiment.fit(X_a_train[commented_mask_train], y_a_train['comment_sentiment'][commented_mask_train])
    print(f"     ✓ Comment sentiment regressor trained ({commented_mask_train.sum()} samples)")
else:
    model_a_sentiment = None
    print("     ⚠ No comments in training data, skipping sentiment model")

# 4. Comment intent classifier (only for commented=True)
print("  4. Training comment intent classifier...")
if commented_mask_train.sum() > 0:
    model_a_intent = XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        random_state=42,
        n_jobs=-1,
        eval_metric='mlogloss'
    )
    model_a_intent.fit(X_a_train[commented_mask_train], y_a_train['comment_intent'][commented_mask_train])
    print(f"     ✓ Comment intent classifier trained ({commented_mask_train.sum()} samples)")
else:
    model_a_intent = None
    print("     ⚠ No comments in training data, skipping intent model")

print("\\n✓ Model A training complete!")

# Store models
model_a = {
    'reaction': model_a_reaction,
    'comment': model_a_comment,
    'sentiment': model_a_sentiment,
    'intent': model_a_intent
}

In [ ]:
# Evaluate Model A
print("=" * 70)
print("MODEL A EVALUATION")
print("=" * 70)

# 1. Reaction type evaluation
print("\\n1. Reaction Type Classification:")
y_reaction_pred = model_a_reaction.predict(X_a_test)
reaction_accuracy = accuracy_score(y_a_test['reaction_type'], y_reaction_pred)
print(f"   Accuracy: {reaction_accuracy:.3f}")
print(f"\\n   Classification Report:")
print(classification_report(y_a_test['reaction_type'], y_reaction_pred, 
                          target_names=sorted(np.unique(y_a_test['reaction_type']))))

# 2. Comment presence evaluation
print("\\n2. Comment Presence Classification:")
y_comment_pred = model_a_comment.predict(X_a_test)
y_comment_proba = model_a_comment.predict_proba(X_a_test)[:, 1]
comment_accuracy = accuracy_score(y_a_test['commented'], y_comment_pred)
comment_auc = roc_auc_score(y_a_test['commented'], y_comment_proba)
print(f"   Accuracy: {comment_accuracy:.3f}")
print(f"   ROC-AUC: {comment_auc:.3f}")
print(f"\\n   Classification Report:")
print(classification_report(y_a_test['commented'], y_comment_pred, 
                          target_names=['No Comment', 'Commented']))

# 3. Comment sentiment evaluation (only for commented samples)
if model_a_sentiment is not None and commented_mask_test.sum() > 0:
    print("\\n3. Comment Sentiment Regression:")
    y_sentiment_pred = model_a_sentiment.predict(X_a_test[commented_mask_test])
    y_sentiment_true = y_a_test['comment_sentiment'][commented_mask_test]
    sentiment_mae = mean_absolute_error(y_sentiment_true, y_sentiment_pred)
    sentiment_rmse = np.sqrt(mean_squared_error(y_sentiment_true, y_sentiment_pred))
    sentiment_r2 = r2_score(y_sentiment_true, y_sentiment_pred)
    print(f"   MAE: {sentiment_mae:.3f}")
    print(f"   RMSE: {sentiment_rmse:.3f}")
    print(f"   R²: {sentiment_r2:.3f}")

# 4. Comment intent evaluation (only for commented samples)
if model_a_intent is not None and commented_mask_test.sum() > 0:
    print("\\n4. Comment Intent Classification:")
    y_intent_pred = model_a_intent.predict(X_a_test[commented_mask_test])
    y_intent_true = y_a_test['comment_intent'][commented_mask_test]
    intent_accuracy = accuracy_score(y_intent_true, y_intent_pred)
    print(f"   Accuracy: {intent_accuracy:.3f}")
    print(f"\\n   Classification Report:")
    print(classification_report(y_intent_true, y_intent_pred, 
                              target_names=sorted(np.unique(y_intent_true))))

print("\\n" + "=" * 70)

In [ ]:
# Visualize Model A performance
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Reaction type confusion matrix
ax1 = axes[0, 0]
reaction_types = sorted(np.unique(y_a_test['reaction_type']))
cm_reaction = confusion_matrix(y_a_test['reaction_type'], y_reaction_pred, labels=reaction_types)
sns.heatmap(cm_reaction, annot=True, fmt='d', cmap='Blues', ax=ax1,
            xticklabels=reaction_types, yticklabels=reaction_types)
ax1.set_xlabel('Predicted', fontsize=12)
ax1.set_ylabel('Actual', fontsize=12)
ax1.set_title('Reaction Type Confusion Matrix', fontsize=14, fontweight='bold')

# 2. Comment presence distribution
ax2 = axes[0, 1]
comment_counts = pd.Series(y_a_test['commented']).value_counts().sort_index()
comment_pred_counts = pd.Series(y_comment_pred).value_counts().sort_index()
x = np.arange(len(comment_counts))
width = 0.35
ax2.bar(x - width/2, comment_counts.values, width, label='Actual', alpha=0.7)
ax2.bar(x + width/2, comment_pred_counts.values, width, label='Predicted', alpha=0.7)
ax2.set_xlabel('Commented', fontsize=12)
ax2.set_ylabel('Count', fontsize=12)
ax2.set_title('Comment Presence: Actual vs Predicted', fontsize=14, fontweight='bold')
ax2.set_xticks(x)
ax2.set_xticklabels(['No', 'Yes'])
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

# 3. Comment sentiment distribution (if available)
ax3 = axes[1, 0]
if model_a_sentiment is not None and commented_mask_test.sum() > 0:
    ax3.scatter(y_sentiment_true, y_sentiment_pred, alpha=0.6, s=50)
    min_val = min(y_sentiment_true.min(), y_sentiment_pred.min())
    max_val = max(y_sentiment_true.max(), y_sentiment_pred.max())
    ax3.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect Prediction')
    ax3.set_xlabel('Actual Sentiment', fontsize=12)
    ax3.set_ylabel('Predicted Sentiment', fontsize=12)
    ax3.set_title(f'Comment Sentiment Prediction (R² = {sentiment_r2:.3f})', fontsize=14, fontweight='bold')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
else:
    ax3.text(0.5, 0.5, 'No comment data available', ha='center', va='center', fontsize=12)
    ax3.set_title('Comment Sentiment Prediction', fontsize=14, fontweight='bold')

# 4. Reaction type distribution by role (sample)
ax4 = axes[1, 1]
# Get a sample of roles
sample_roles = df_individual['commenter_role'].value_counts().head(5).index
role_reaction_data = []
for role in sample_roles:
    role_mask = df_individual['commenter_role'] == role
    reactions = df_individual[role_mask]['reaction_type'].value_counts()
    for reaction_type, count in reactions.items():
        role_reaction_data.append({'role': role, 'reaction': reaction_type, 'count': count})
role_reaction_df = pd.DataFrame(role_reaction_data)
if len(role_reaction_df) > 0:
    role_reaction_pivot = role_reaction_df.pivot(index='role', columns='reaction', values='count').fillna(0)
    role_reaction_pivot.plot(kind='bar', stacked=True, ax=ax4, colormap='Set3')
    ax4.set_xlabel('Role', fontsize=12)
    ax4.set_ylabel('Count', fontsize=12)
    ax4.set_title('Reaction Distribution by Role (Sample)', fontsize=14, fontweight='bold')
    ax4.legend(title='Reaction Type', bbox_to_anchor=(1.05, 1), loc='upper left')
    ax4.tick_params(axis='x', rotation=45)
else:
    ax4.text(0.5, 0.5, 'No role data available', ha='center', va='center', fontsize=12)
    ax4.set_title('Reaction Distribution by Role', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

### Model B: Post-Level Aggregation & Impact Model

Model B aggregates Model A predictions to forecast overall post performance.

In [ ]:
def aggregate_model_a_predictions(
    post_text: str,
    post_metadata: Dict,
    role_archetypes: List[str],
    model_a: Dict,
    feature_info_a: Dict,
    generate_embeddings_func
) -> Dict:
    """
    Use Model A to predict reactions for different role archetypes, then aggregate.
    
    Returns aggregated features for Model B:
    - Reaction counts by type
    - Comment probability by role
    - Average sentiment by role
    - Role distribution of engagement
    """
    # Prepare features for each role archetype
    predictions_by_role = {}
    
    # Get post embedding
    post_text_clean = post_text.lower().strip()
    if post_text in feature_info_a['text_embeddings_dict']:
        post_embedding = feature_info_a['text_embeddings_dict'][post_text]
    else:
        post_embedding = generate_embeddings_func([post_text_clean])[0]
    
    # Prepare post features (same for all roles)
    le_post_topic = feature_info_a['encoders']['post_topic']
    le_post_author = feature_info_a['encoders']['post_author']
    le_time = feature_info_a['encoders']['time']
    ohe_post = feature_info_a['one_hot_encoders']['post']
    scaler_post = feature_info_a['scaler_post_numerical']
    
    # Encode post metadata
    post_topic_encoded = le_post_topic.transform([post_metadata.get('topic', 'general')])[0]
    post_author_encoded = le_post_author.transform([post_metadata.get('author_role', 'CEO')])[0]
    time_encoded = le_time.transform([post_metadata.get('time_posted', 'morning')])[0]
    
    post_categorical = ohe_post.transform([[
        post_metadata.get('topic', 'general'),
        post_metadata.get('author_role', 'CEO'),
        post_metadata.get('time_posted', 'morning')
    ]])
    
    post_length = post_metadata.get('post_length', len(post_text.split()))
    has_hashtags = post_metadata.get('hashtags', '#' in post_text)
    has_mentions = post_metadata.get('mentions', '@' in post_text)
    has_urls = post_metadata.get('urls_present', 'http' in post_text.lower())
    
    post_numerical = scaler_post.transform([[post_length, has_hashtags, has_mentions, has_urls]])
    
    # Predict for each role
    le_role = feature_info_a['encoders']['role']
    le_industry = feature_info_a['encoders']['industry']
    le_seniority = feature_info_a['encoders']['seniority']
    ohe_person = feature_info_a['one_hot_encoders']['person']
    
    # Role archetype mappings (simplified - in practice, use actual data)
    role_industry_map = {
        'Software Engineer': 'Tech', 'Product Manager': 'Tech', 'Data Scientist': 'Tech',
        'CEO': 'Tech', 'CTO': 'Tech', 'Founder': 'Tech', 'Recruiter': 'Tech',
        'Marketing Director': 'Tech', 'Designer': 'Tech', 'Consultant': 'Consulting',
        'Student': 'Education'
    }
    role_seniority_map = {
        'Software Engineer': 'Mid', 'Product Manager': 'Mid', 'Data Scientist': 'Mid',
        'CEO': 'Exec', 'CTO': 'Exec', 'Founder': 'Exec', 'Recruiter': 'Mid',
        'Marketing Director': 'Senior', 'Designer': 'Mid', 'Consultant': 'Senior',
        'Student': 'Junior'
    }
    
    for role in role_archetypes:
        try:
            # Encode person features
            industry = role_industry_map.get(role, 'Tech')
            seniority = role_seniority_map.get(role, 'Mid')
            
            person_categorical = ohe_person.transform([[role, industry, seniority]])
            
            # Combine features
            X_role = np.hstack([
                post_embedding.reshape(1, -1),
                post_categorical,
                post_numerical,
                person_categorical
            ])
            
            # Make predictions
            reaction_pred = model_a['reaction'].predict(X_role)[0]
            reaction_proba = model_a['reaction'].predict_proba(X_role)[0]
            comment_pred = model_a['comment'].predict(X_role)[0]
            comment_proba = model_a['comment'].predict_proba(X_role)[0, 1]
            
            sentiment_pred = 0.0
            intent_pred = 'none'
            if comment_pred == 1 and model_a['sentiment'] is not None:
                sentiment_pred = model_a['sentiment'].predict(X_role)[0]
                if model_a['intent'] is not None:
                    intent_pred = model_a['intent'].predict(X_role)[0]
            
            predictions_by_role[role] = {
                'reaction_type': reaction_pred,
                'reaction_proba': reaction_proba,
                'commented': comment_pred,
                'comment_proba': comment_proba,
                'sentiment': sentiment_pred,
                'intent': intent_pred
            }
        except Exception as e:
            # Skip roles not in training data
            continue
    
    # Aggregate predictions
    if len(predictions_by_role) == 0:
        # Return defaults if no predictions
        return {
            'pct_engineers_reacting': 0.0,
            'pct_pms_commenting': 0.0,
            'pct_founders_engaging': 0.0,
            'avg_sentiment_per_role': {},
            'predicted_reaction_distribution': {},
            'predicted_comment_volume': 0.0
        }
    
    # Calculate role-specific metrics
    engineer_roles = ['Software Engineer', 'Data Scientist', 'CTO']
    pm_roles = ['Product Manager']
    founder_roles = ['Founder', 'CEO']
    
    engineers_reacting = sum(1 for r in engineer_roles 
                            if r in predictions_by_role and predictions_by_role[r]['reaction_type'] != 'none')
    pct_engineers_reacting = engineers_reacting / len(engineer_roles) if engineer_roles else 0.0
    
    pms_commenting = sum(1 for r in pm_roles 
                        if r in predictions_by_role and predictions_by_role[r]['commented'] == 1)
    pct_pms_commenting = pms_commenting / len(pm_roles) if pm_roles else 0.0
    
    founders_engaging = sum(1 for r in founder_roles 
                           if r in predictions_by_role and predictions_by_role[r]['reaction_type'] != 'none')
    pct_founders_engaging = founders_engaging / len(founder_roles) if founder_roles else 0.0
    
    # Average sentiment by role
    avg_sentiment_per_role = {}
    for role, preds in predictions_by_role.items():
        if preds['commented'] == 1:
            avg_sentiment_per_role[role] = preds['sentiment']
    
    # Reaction distribution
    reaction_counts = {}
    for preds in predictions_by_role.values():
        rt = preds['reaction_type']
        reaction_counts[rt] = reaction_counts.get(rt, 0) + 1
    total_reactions = sum(reaction_counts.values())
    predicted_reaction_distribution = {k: v/total_reactions if total_reactions > 0 else 0 
                                      for k, v in reaction_counts.items()}
    
    # Predicted comment volume (sum of comment probabilities)
    predicted_comment_volume = sum(preds['comment_proba'] for preds in predictions_by_role.values())
    
    return {
        'pct_engineers_reacting': pct_engineers_reacting,
        'pct_pms_commenting': pct_pms_commenting,
        'pct_founders_engaging': pct_founders_engaging,
        'avg_sentiment_per_role': avg_sentiment_per_role,
        'predicted_reaction_distribution': predicted_reaction_distribution,
        'predicted_comment_volume': predicted_comment_volume
    }

print("✓ Aggregation function ready")

In [ ]:
def prepare_model_b_features(
    post_level_df: pd.DataFrame,
    model_a: Dict,
    feature_info_a: Dict,
    generate_embeddings_func
) -> Tuple[np.ndarray, np.ndarray, Dict]:
    """
    Prepare features for Model B (post-level predictions).
    
    Features:
    - Post text embeddings
    - Post metadata
    - Aggregated Model A predictions (role distributions, engagement patterns)
    
    Returns:
    - X_b: Feature matrix
    - y_b: Target variables (total reactions, comment volume, role distribution, engagement quality)
    - feature_info_b: Feature information
    """
    print("Preparing Model B features...")
    print("This may take a while as we aggregate Model A predictions for each post...")
    
    # Get unique posts
    unique_posts = post_level_df[['post_text']].drop_duplicates()
    unique_posts['post_text_clean'] = unique_posts['post_text'].str.lower().str.strip()
    
    # Generate embeddings for unique posts
    print(f"Generating embeddings for {len(unique_posts)} unique posts...")
    post_embeddings = generate_embeddings(unique_posts['post_text_clean'].tolist())
    text_embeddings_dict_b = dict(zip(unique_posts['post_text'], post_embeddings))
    
    # Get all role archetypes from training data
    role_archetypes = sorted(df_individual['commenter_role'].unique().tolist())
    
    # For each post, aggregate Model A predictions
    aggregated_features_list = []
    post_features_list = []
    targets_list = []
    
    for idx, post_row in post_level_df.iterrows():
        post_text = post_row['post_text']
        
        # Get post embedding
        post_embedding = text_embeddings_dict_b[post_text]
        
        # Prepare post metadata
        post_topic = 'AI' if 'AI' in post_text or 'artificial intelligence' in post_text.lower() else \
                     'startups' if 'startup' in post_text.lower() else \
                     'technology' if 'tech' in post_text.lower() else 'general'
        
        post_metadata = {
            'topic': post_topic,
            'author_role': post_row.get('job_role', 'CEO'),
            'time_posted': post_row.get('time_window', 'morning'),
            'post_length': len(post_text.split()),
            'hashtags': '#' in post_text,
            'mentions': '@' in post_text,
            'urls_present': 'http' in post_text.lower()
        }
        
        # Aggregate Model A predictions
        aggregated = aggregate_model_a_predictions(
            post_text=post_text,
            post_metadata=post_metadata,
            role_archetypes=role_archetypes,
            model_a=model_a,
            feature_info_a=feature_info_a,
            generate_embeddings_func=generate_embeddings_func
        )
        
        # Flatten aggregated features
        aggregated_features = [
            aggregated['pct_engineers_reacting'],
            aggregated['pct_pms_commenting'],
            aggregated['pct_founders_engaging'],
            aggregated['predicted_comment_volume']
        ]
        
        # Add reaction distribution features
        reaction_types = ['like', 'love', 'insightful', 'celebrate', 'none']
        for rt in reaction_types:
            aggregated_features.append(aggregated['predicted_reaction_distribution'].get(rt, 0.0))
        
        # Add average sentiment (flatten dict)
        role_sentiments = aggregated['avg_sentiment_per_role']
        for role in role_archetypes[:5]:  # Top 5 roles
            aggregated_features.append(role_sentiments.get(role, 0.0))
        
        aggregated_features_list.append(aggregated_features)
        post_features_list.append(post_embedding)
        
        # Calculate targets from individual-level data for this post
        post_individual = df_individual[df_individual['post_text'] == post_text]
        if len(post_individual) > 0:
            total_reactions = len(post_individual[post_individual['reaction_type'] != 'none'])
            comment_volume = post_individual['commented'].sum()
            
            # Role distribution (simplified - percentage of each role)
            role_counts = post_individual['commenter_role'].value_counts()
            total_people = len(post_individual)
            engineer_pct = role_counts[role_counts.index.isin(['Software Engineer', 'Data Scientist', 'CTO'])].sum() / total_people if total_people > 0 else 0
            pm_pct = role_counts[role_counts.index.isin(['Product Manager'])].sum() / total_people if total_people > 0 else 0
            founder_pct = role_counts[role_counts.index.isin(['Founder', 'CEO'])].sum() / total_people if total_people > 0 else 0
            
            # Engagement quality score (simple heuristic: reactions + comments + positive sentiment)
            avg_sentiment = post_individual[post_individual['commented'] == 1]['comment_sentiment'].mean() if post_individual['commented'].sum() > 0 else 0
            engagement_quality = (total_reactions / len(post_individual)) * 0.4 + \
                               (comment_volume / len(post_individual)) * 0.4 + \
                               (max(0, avg_sentiment)) * 0.2
            
            targets_list.append([
                total_reactions,
                comment_volume,
                engineer_pct,
                pm_pct,
                founder_pct,
                engagement_quality
            ])
        else:
            targets_list.append([0, 0, 0, 0, 0, 0])
    
    # Combine features
    post_embeddings_array = np.array(post_features_list)
    aggregated_features_array = np.array(aggregated_features_list)
    
    X_b = np.hstack([post_embeddings_array, aggregated_features_array])
    y_b = np.array(targets_list)
    
    feature_info_b = {
        'embedding_dim': post_embeddings_array.shape[1],
        'aggregated_dim': aggregated_features_array.shape[1],
        'total_dim': X_b.shape[1],
        'text_embeddings_dict': text_embeddings_dict_b
    }
    
    print(f"✓ Feature matrix shape: {X_b.shape}")
    print(f"  - Post embeddings: {post_embeddings_array.shape[1]} dims")
    print(f"  - Aggregated Model A features: {aggregated_features_array.shape[1]} dims")
    print(f"\\nTarget variables: total_reactions, comment_volume, engineer_pct, pm_pct, founder_pct, engagement_quality")
    
    return X_b, y_b, feature_info_b

# Prepare Model B features
X_b, y_b, feature_info_b = prepare_model_b_features(df, model_a, feature_info_a, generate_embeddings)
print(f"\\nTarget matrix shape: {y_b.shape}")

In [ ]:
# Split data for Model B
X_b_train, X_b_test, y_b_train, y_b_test = train_test_split(
    X_b, y_b, test_size=0.2, random_state=42
)

print(f"Model B Training set: {X_b_train.shape[0]} samples")
print(f"Model B Test set: {X_b_test.shape[0]} samples")

# Train Model B
print("\\nTraining Model B (post-level aggregation model)...")

base_model_b = XGBRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

model_b = MultiOutputRegressor(base_model_b)

model_b.fit(X_b_train, y_b_train)
print("✓ Model B training complete")

# Make predictions
y_b_train_pred = model_b.predict(X_b_train)
y_b_test_pred = model_b.predict(X_b_test)

print(f"\\nPredictions shape: {y_b_test_pred.shape}")

# Target names for Model B
target_names_b = ['total_reactions', 'comment_volume', 'engineer_pct', 'pm_pct', 'founder_pct', 'engagement_quality']

In [ ]:
# Evaluate Model B
print("=" * 70)
print("MODEL B EVALUATION")
print("=" * 70)

# Calculate metrics for each target
b_metrics = []
for i, target_name in enumerate(target_names_b):
    y_true_i = y_b_test[:, i]
    y_pred_i = y_b_test_pred[:, i]
    
    mae = mean_absolute_error(y_true_i, y_pred_i)
    rmse = np.sqrt(mean_squared_error(y_true_i, y_pred_i))
    r2 = r2_score(y_true_i, y_pred_i)
    
    b_metrics.append({
        'target': target_name,
        'MAE': mae,
        'RMSE': rmse,
        'R²': r2
    })

b_metrics_df = pd.DataFrame(b_metrics)
print("\\nTest Set Metrics:")
print(b_metrics_df.to_string(index=False))

# Visualize Model B performance
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for i, target_name in enumerate(target_names_b):
    ax = axes[i]
    y_true_i = y_b_test[:, i]
    y_pred_i = y_b_test_pred[:, i]
    
    ax.scatter(y_true_i, y_pred_i, alpha=0.6, s=50)
    min_val = min(y_true_i.min(), y_pred_i.min())
    max_val = max(y_true_i.max(), y_pred_i.max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect Prediction')
    
    r2 = r2_score(y_true_i, y_pred_i)
    ax.set_xlabel(f'Actual {target_name}')
    ax.set_ylabel(f'Predicted {target_name}')
    ax.set_title(f'{target_name}\\nR² = {r2:.3f}')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### End-to-End Prediction Pipeline

Combine Model A and Model B for complete post engagement prediction.

In [ ]:
def predict_post_engagement_pipeline(
    post_text: str,
    post_metadata: Dict,
    model_a: Dict,
    model_b: MultiOutputRegressor,
    feature_info_a: Dict,
    feature_info_b: Dict,
    generate_embeddings_func
) -> Dict:
    """
    End-to-end prediction pipeline: Model A → Aggregate → Model B.
    
    Args:
        post_text: The post text to analyze
        post_metadata: Dictionary with post metadata (topic, author_role, time_posted, etc.)
        model_a: Trained Model A dictionary
        model_b: Trained Model B (MultiOutputRegressor)
        feature_info_a: Feature info from Model A training
        feature_info_b: Feature info from Model B training
        generate_embeddings_func: Function to generate embeddings
    
    Returns:
        Dictionary with:
        - individual_predictions: Model A predictions by role
        - aggregated_features: Aggregated Model A features
        - post_level_predictions: Model B predictions
    """
    # Step 1: Get role archetypes
    role_archetypes = sorted(df_individual['commenter_role'].unique().tolist())
    
    # Step 2: Use Model A to predict for each role
    aggregated = aggregate_model_a_predictions(
        post_text=post_text,
        post_metadata=post_metadata,
        role_archetypes=role_archetypes,
        model_a=model_a,
        feature_info_a=feature_info_a,
        generate_embeddings_func=generate_embeddings_func
    )
    
    # Step 3: Prepare features for Model B
    post_text_clean = post_text.lower().strip()
    
    # Get or generate post embedding
    if post_text in feature_info_b['text_embeddings_dict']:
        post_embedding = feature_info_b['text_embeddings_dict'][post_text]
    else:
        post_embedding = generate_embeddings_func([post_text_clean])[0]
    
    # Flatten aggregated features (same as in training)
    aggregated_features = [
        aggregated['pct_engineers_reacting'],
        aggregated['pct_pms_commenting'],
        aggregated['pct_founders_engaging'],
        aggregated['predicted_comment_volume']
    ]
    
    # Add reaction distribution
    reaction_types = ['like', 'love', 'insightful', 'celebrate', 'none']
    for rt in reaction_types:
        aggregated_features.append(aggregated['predicted_reaction_distribution'].get(rt, 0.0))
    
    # Add average sentiment
    role_sentiments = aggregated['avg_sentiment_per_role']
    for role in role_archetypes[:5]:
        aggregated_features.append(role_sentiments.get(role, 0.0))
    
    # Combine features
    X_b_input = np.hstack([post_embedding.reshape(1, -1), np.array([aggregated_features])])
    
    # Step 4: Use Model B to predict post-level metrics
    post_level_pred = model_b.predict(X_b_input)[0]
    
    return {
        'individual_predictions': aggregated,
        'aggregated_features': aggregated_features,
        'post_level_predictions': {
            'total_reactions': post_level_pred[0],
            'comment_volume': post_level_pred[1],
            'engineer_pct': post_level_pred[2],
            'pm_pct': post_level_pred[3],
            'founder_pct': post_level_pred[4],
            'engagement_quality': post_level_pred[5]
        }
    }

print("✓ End-to-end prediction pipeline ready")

In [ ]:
# Test the pipeline with a sample post
test_post = "Excited to share our latest AI breakthrough! This technology will revolutionize how we work."
test_metadata = {
    'topic': 'AI',
    'author_role': 'CEO',
    'time_posted': 'morning',
    'post_length': len(test_post.split()),
    'hashtags': False,
    'mentions': False,
    'urls_present': False
}

print("Testing end-to-end pipeline...")
print(f"Post: {test_post}")
print("\\n" + "=" * 70)

result = predict_post_engagement_pipeline(
    post_text=test_post,
    post_metadata=test_metadata,
    model_a=model_a,
    model_b=model_b,
    feature_info_a=feature_info_a,
    feature_info_b=feature_info_b,
    generate_embeddings_func=generate_embeddings
)

print("\\nINDIVIDUAL-LEVEL PREDICTIONS (Model A):")
print(f"  Engineers reacting: {result['individual_predictions']['pct_engineers_reacting']:.1%}")
print(f"  PMs commenting: {result['individual_predictions']['pct_pms_commenting']:.1%}")
print(f"  Founders engaging: {result['individual_predictions']['pct_founders_engaging']:.1%}")
print(f"  Predicted comment volume: {result['individual_predictions']['predicted_comment_volume']:.1f}")
print(f"\\n  Reaction distribution:")
for reaction, prob in result['individual_predictions']['predicted_reaction_distribution'].items():
    print(f"    {reaction}: {prob:.1%}")

print("\\n" + "=" * 70)
print("POST-LEVEL PREDICTIONS (Model B):")
for key, value in result['post_level_predictions'].items():
    if 'pct' in key:
        print(f"  {key}: {value:.1%}")
    elif 'quality' in key:
        print(f"  {key}: {value:.3f}")
    else:
        print(f"  {key}: {value:.1f}")
print("=" * 70)

In [ ]:
# Visualize the pipeline flow and results
fig = plt.figure(figsize=(16, 10))

# Create a 2x2 grid
gs = fig.add_gridspec(2, 2, hspace=0.3, wspace=0.3)

# 1. Role-based reaction predictions (Model A output)
ax1 = fig.add_subplot(gs[0, 0])
reaction_dist = result['individual_predictions']['predicted_reaction_distribution']
reactions = list(reaction_dist.keys())
probs = list(reaction_dist.values())
colors_map = {'like': '#3498db', 'love': '#e74c3c', 'insightful': '#2ecc71', 'celebrate': '#f39c12', 'none': '#95a5a6'}
colors = [colors_map.get(r, '#95a5a6') for r in reactions]
ax1.bar(reactions, probs, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
ax1.set_ylabel('Probability', fontsize=12)
ax1.set_title('Model A: Predicted Reaction Distribution', fontsize=13, fontweight='bold')
ax1.set_ylim(0, max(probs) * 1.2)
ax1.grid(axis='y', alpha=0.3)
for i, (r, p) in enumerate(zip(reactions, probs)):
    ax1.text(i, p, f'{p:.1%}', ha='center', va='bottom', fontsize=10, fontweight='bold')

# 2. Role engagement percentages
ax2 = fig.add_subplot(gs[0, 1])
role_metrics = {
    'Engineers': result['individual_predictions']['pct_engineers_reacting'],
    'PMs': result['individual_predictions']['pct_pms_commenting'],
    'Founders': result['individual_predictions']['pct_founders_engaging']
}
roles = list(role_metrics.keys())
values = list(role_metrics.values())
ax2.barh(roles, values, color=['#3498db', '#2ecc71', '#9b59b6'], alpha=0.7, edgecolor='black', linewidth=1.5)
ax2.set_xlabel('Engagement Percentage', fontsize=12)
ax2.set_title('Model A: Role Engagement Rates', fontsize=13, fontweight='bold')
ax2.set_xlim(0, 1)
ax2.grid(axis='x', alpha=0.3)
for i, (role, val) in enumerate(zip(roles, values)):
    ax2.text(val, i, f'{val:.1%}', va='center', ha='left', fontsize=11, fontweight='bold', 
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# 3. Post-level predictions (Model B output)
ax3 = fig.add_subplot(gs[1, 0])
post_preds = result['post_level_predictions']
metrics_to_plot = {
    'Total Reactions': post_preds['total_reactions'],
    'Comment Volume': post_preds['comment_volume'],
    'Engagement Quality': post_preds['engagement_quality'] * 100  # Scale for visibility
}
metric_names = list(metrics_to_plot.keys())
metric_values = list(metrics_to_plot.values())
colors_metrics = ['#e74c3c', '#3498db', '#2ecc71']
bars = ax3.bar(metric_names, metric_values, color=colors_metrics, alpha=0.7, edgecolor='black', linewidth=1.5)
ax3.set_ylabel('Value', fontsize=12)
ax3.set_title('Model B: Post-Level Predictions', fontsize=13, fontweight='bold')
ax3.grid(axis='y', alpha=0.3)
for bar, val in zip(bars, metric_values):
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height,
             f'{val:.1f}', ha='center', va='bottom', fontsize=11, fontweight='bold')
ax3.tick_params(axis='x', rotation=45)

# 4. Role distribution (Model B output)
ax4 = fig.add_subplot(gs[1, 1])
role_dist = {
    'Engineers': post_preds['engineer_pct'],
    'PMs': post_preds['pm_pct'],
    'Founders': post_preds['founder_pct']
}
role_names = list(role_dist.keys())
role_values = list(role_dist.values())
# Add "Others" to make it sum to 1
others = 1 - sum(role_values)
if others > 0:
    role_names.append('Others')
    role_values.append(others)
colors_pie = ['#3498db', '#2ecc71', '#9b59b6', '#95a5a6']
ax4.pie(role_values, labels=role_names, autopct='%1.1f%%', colors=colors_pie[:len(role_values)],
        startangle=90, textprops={'fontsize': 11, 'fontweight': 'bold'})
ax4.set_title('Model B: Predicted Role Distribution', fontsize=13, fontweight='bold')

plt.suptitle('Two-Model Pipeline: Model A → Model B Predictions', fontsize=16, fontweight='bold', y=0.98)
plt.show()

### Summary: Two-Model Architecture

**Key Benefits:**

1. **Interpretability**: Model A shows which roles react how, making predictions explainable
2. **Granularity**: Individual-level predictions can be aggregated in different ways
3. **Flexibility**: Can adjust role archetypes or add new ones without retraining Model B
4. **Scalability**: Model A predictions can be cached and reused for multiple posts

**Model A** learns: "How do different role archetypes react to posts?"
**Model B** learns: "Which posts generate valuable engagement patterns?"

**Pipeline**: Post → Model A (role predictions) → Aggregate → Model B → Post-level metrics

## Test Your Draft Post

Use this section to test how your draft post might perform! Simply paste your post text below and optionally adjust the metadata. The model will predict engagement metrics based on the trained patterns.

**Instructions:**
1. Paste your draft post text in the `test_post_text` variable below
2. Optionally adjust persona and context metadata (or leave as None to use defaults from training data)
3. Run all cells in this section to see predictions

In [ ]:
test_post_text = (
    "I've seen the chatter. It’s okay. Conversations like this usually come from people who care about the same ecosystem we do.\n\n"
    "The message I shared recently was meant for founders and builders in Australia. The point wasn’t about working till 11:30pm. "
    "It was about ambition and what it takes to close the gap between what’s possible here and what’s already happening in the world.\n\n"
    "Lyra was built for people who want to build. Many of our engineers dream of starting companies, and our role is to create a support system "
    "with revenue, environment, and a community so they can pursue these ambitions (with the highest chance of success).\n\n"
    "I believe drive looks different for everyone. Some want balance, some want to push limits. Both belong here.\n\n"
    "Im proud of what Lyra's building. From engineers mentoring each other weekly, to clients trusting us with complex products, "
    "to side projects turning into startups.\n\n"
    "Im grateful to everyone who’s part of this journey. The mission stays the same: build, teach, and grow."
)

In [ ]:
# Paste your draft post text here
# test_post_text = "Excited to share my latest insights on AI! This technology is revolutionizing the industry and changing how we work."

# Optional: Persona metadata (set to None to use defaults from training data)
test_job_role = None  # e.g., "Software Engineer", "Product Manager", "CEO"
test_affiliation = None  # e.g., "Tech Corp", "StartupXYZ", "BigTech Inc"
test_account_age = None  # Account age in days (e.g., 365 for 1 year)

# Optional: Context metadata (set to None to use defaults from training data)
test_audience_size = None  # Typical audience size (e.g., 10000)
test_baseline_engagement = None  # Baseline engagement rate (e.g., 0.05 for 5%)
test_time_window = None  # Time window: "morning", "afternoon", "evening", "weekend"

# Calculate defaults from training data if not provided
if test_job_role is None:
    test_job_role = df['job_role'].mode()[0] if len(df) > 0 else "Software Engineer"
if test_affiliation is None:
    test_affiliation = df['affiliation'].mode()[0] if len(df) > 0 else "Tech Corp"
if test_account_age is None:
    test_account_age = int(df['account_age'].median()) if len(df) > 0 else 365
if test_audience_size is None:
    test_audience_size = int(df['audience_size'].median()) if len(df) > 0 else 10000
if test_baseline_engagement is None:
    test_baseline_engagement = float(df['baseline_engagement'].median()) if len(df) > 0 else 0.05
if test_time_window is None:
    test_time_window = df['time_window'].mode()[0] if len(df) > 0 else "morning"

print("Test Configuration:")
print(f"  Post text: {test_post_text[:100]}..." if len(test_post_text) > 100 else f"  Post text: {test_post_text}")
print(f"  Job role: {test_job_role}")
print(f"  Affiliation: {test_affiliation}")
print(f"  Account age: {test_account_age} days")
print(f"  Audience size: {test_audience_size:,}")
print(f"  Baseline engagement: {test_baseline_engagement:.2%}")
print(f"  Time window: {test_time_window}")

In [ ]:
def prepare_single_post_features(
    post_text: str,
    job_role: str,
    affiliation: str,
    account_age: int,
    audience_size: int,
    baseline_engagement: float,
    time_window: str,
    feature_info: Dict,
    generate_embeddings_func
) -> np.ndarray:
    """
    Prepare features for a single post using the same preprocessing pipeline as training.
    
    Args:
        post_text: The post text to analyze
        job_role: Job role of the poster
        affiliation: Affiliation of the poster
        account_age: Account age in days
        audience_size: Typical audience size
        baseline_engagement: Baseline engagement rate
        time_window: Time window when post is published
        feature_info: Dictionary containing encoders and scalers from training
        generate_embeddings_func: Function to generate embeddings
    
    Returns:
        Feature vector ready for model prediction
    """
    # Clean and normalize text
    post_text_clean = post_text.lower().strip()
    
    # Generate embeddings for the post text
    text_embeddings = generate_embeddings_func([post_text_clean])
    
    # Prepare metadata as DataFrame for encoding
    metadata_df = pd.DataFrame({
        'job_role': [job_role],
        'affiliation': [affiliation],
        'time_window': [time_window],
        'account_age': [account_age],
        'audience_size': [audience_size],
        'baseline_engagement': [baseline_engagement]
    })
    
    # Handle unseen categorical values
    ohe = feature_info['one_hot_encoder']
    try:
        # Try to encode with existing encoder
        categorical_features = ohe.transform(metadata_df[['job_role', 'affiliation', 'time_window']])
    except ValueError as e:
        # Handle unseen categories by using most common values from training
        print(f"Warning: Unseen category detected. Using defaults from training data.")
        # Get most common values from training data
        default_job = df['job_role'].mode()[0]
        default_affiliation = df['affiliation'].mode()[0]
        default_time = df['time_window'].mode()[0]
        
        metadata_df['job_role'] = [default_job if job_role not in df['job_role'].values else job_role]
        metadata_df['affiliation'] = [default_affiliation if affiliation not in df['affiliation'].values else affiliation]
        metadata_df['time_window'] = [default_time if time_window not in df['time_window'].values else time_window]
        
        categorical_features = ohe.transform(metadata_df[['job_role', 'affiliation', 'time_window']])
    
    # Normalize numerical features
    scaler = feature_info['scaler']
    numerical_features = scaler.transform(
        metadata_df[['account_age', 'audience_size', 'baseline_engagement']]
    )
    
    # Combine all features: embeddings + categorical + numerical
    X = np.hstack([
        text_embeddings,
        categorical_features,
        numerical_features
    ])
    
    return X

print("✓ Preprocessing function ready")

In [ ]:
# Check if model and feature_info are available
try:
    if 'model' not in globals() or 'feature_info' not in globals():
        raise ValueError("Model not trained yet. Please run the training cells first.")
    
    # Prepare features for the test post
    print("Preprocessing test post...")
    X_test_post = prepare_single_post_features(
        post_text=test_post_text,
        job_role=test_job_role,
        affiliation=test_affiliation,
        account_age=test_account_age,
        audience_size=test_audience_size,
        baseline_engagement=test_baseline_engagement,
        time_window=test_time_window,
        feature_info=feature_info,
        generate_embeddings_func=generate_embeddings
    )
    
    print(f"✓ Features prepared: {X_test_post.shape}")
    
    # Make predictions
    print("\\nMaking predictions...")
    predictions = model.predict(X_test_post)
    
    # Extract individual predictions
    pct_positive = predictions[0][0]
    pct_negative = predictions[0][1]
    comment_sentiment_dist = predictions[0][2]
    engagement_velocity = predictions[0][3]
    
    # Clip values to valid ranges
    pct_positive = np.clip(pct_positive, 0, 100)
    pct_negative = np.clip(pct_negative, 0, 100)
    comment_sentiment_dist = np.clip(comment_sentiment_dist, -1, 1)
    engagement_velocity = np.clip(engagement_velocity, 0, 1)
    
    print("✓ Predictions complete")
    
except Exception as e:
    print(f"Error: {str(e)}")
    print("\\nMake sure you have:")
    print("  1. Run all cells up to and including the model training section")
    print("  2. The 'model' and 'feature_info' variables are available")
    raise

In [ ]:
# Display results in a formatted way
print("=" * 70)
print("PREDICTED ENGAGEMENT METRICS")
print("=" * 70)
print()

# Format and display each metric
print(f"📈 % Positive Reactions: {pct_positive:.1f}%")
if pct_positive >= 40:
    print("   → Excellent! This post is predicted to receive strong positive engagement.")
elif pct_positive >= 25:
    print("   → Good! This post should perform well with positive reactions.")
elif pct_positive >= 15:
    print("   → Moderate positive engagement expected.")
else:
    print("   → Lower positive engagement predicted. Consider refining the content.")
print()

print(f"📉 % Negative Reactions: {pct_negative:.1f}%")
if pct_negative <= 5:
    print("   → Very low negative reactions - great!")
elif pct_negative <= 10:
    print("   → Low negative reactions - acceptable.")
elif pct_negative <= 20:
    print("   → Moderate negative reactions - some risk of controversy.")
else:
    print("   → High negative reactions predicted. Content may be controversial.")
print()

print(f"💬 Comment Sentiment Distribution: {comment_sentiment_dist:.3f}")
print("   (Range: -1 = very negative, 0 = neutral, +1 = very positive)")
if comment_sentiment_dist >= 0.5:
    print("   → Comments are predicted to be very positive!")
elif comment_sentiment_dist >= 0.2:
    print("   → Comments should be generally positive.")
elif comment_sentiment_dist >= -0.2:
    print("   → Comments expected to be neutral to mixed.")
elif comment_sentiment_dist >= -0.5:
    print("   → Comments may be somewhat negative.")
else:
    print("   → Comments predicted to be negative. Consider revising.")
print()

print(f"⚡ Engagement Velocity: {engagement_velocity:.3f}")
print("   (Range: 0 = slow/late engagement, 1 = fast/early engagement)")
if engagement_velocity >= 0.7:
    print("   → High velocity! Engagement is predicted to happen quickly.")
elif engagement_velocity >= 0.5:
    print("   → Moderate velocity - steady engagement expected.")
elif engagement_velocity >= 0.3:
    print("   → Lower velocity - engagement may build gradually.")
else:
    print("   → Slow velocity - engagement may take time to develop.")
print()

print("=" * 70)

## Compare Predictions with Actual Results

Analyze how the model's predictions compare to the actual engagement from real comments.

In [ ]:
# Analyze actual comments from the post
# Based on the comments provided, categorize them

actual_comments_analysis = {
    'positive': [
        "Ankit Talwar - Supportive, understands the context",
        "Humza Rafiq - Love this perspective, positive",
        "Brenton Andersen - Supportive of ambition",
        "Smit Bhatt - Talent grows fastest when culture doesn't shame ambition",
        "Ryan Zahrai - Strong support, defends founder culture",
        "Simon Liu - Supporting you, understands timezones",
        "Harry M.D. Nguyen - Supports Lyra's culture and mission",
        "Shubham P. - Extraordinary goals require extraordinary efforts",
        "Edward J Roland III - Keep grinding",
        "Jerry X. - Bucket list moment",
        "Jack Ryan - Supportive, understands enjoyment of work",
        "Ricki Burke - Supportive, not everyone suited for every company",
        "Frantz Rigaud - Supportive of impact-focused ambition",
        "Anh Dao (author reply) - Agrees with sentiment"
    ],
    'negative': [
        "Cameron Craig - 'You have torched your career in one post'",
        "Adam R. - Concerned about worker safety and fatigue",
        "Adam Heathcote - 'You didn't make that clear and now you are backpeddling'",
        "Richard Forsythe - 'Terrible post. Terrible outlook. Terrible company.'",
        "Daniel Pyemont - 'LOL this is hilarious' (sarcastic)",
        "Avi Ben-Moyal - Negative about applicants after 'shit show'"
    ],
    'mixed_neutral': [
        "Simon Monk - Balanced: true for founders but risk of burnout",
        "Josh O'Dea - Concerned about vulnerable employees (high school grad)",
        "Chris Rickard - Questioning: equity vs contractors working late"
    ]
}

# Count comments
n_positive = len(actual_comments_analysis['positive'])
n_negative = len(actual_comments_analysis['negative'])
n_mixed = len(actual_comments_analysis['mixed_neutral'])
n_total = n_positive + n_negative + n_mixed

# Calculate percentages
actual_pct_positive = (n_positive / n_total * 100) if n_total > 0 else 0
actual_pct_negative = (n_negative / n_total * 100) if n_total > 0 else 0
actual_pct_mixed = (n_mixed / n_total * 100) if n_total > 0 else 0

# Estimate sentiment distribution (positive comments = +1, negative = -1, mixed = 0)
actual_sentiment = (n_positive - n_negative) / n_total if n_total > 0 else 0

# Engagement velocity: Based on comment timing (most comments within 3-4 weeks suggests moderate velocity)
# Comments appeared over 3-4 weeks, so moderate velocity
actual_velocity = 0.4  # Estimated based on comment spread

print("ACTUAL ENGAGEMENT ANALYSIS")
print("=" * 70)
print(f"Total comments analyzed: {n_total}")
print(f"  Positive: {n_positive} ({actual_pct_positive:.1f}%)")
print(f"  Negative: {n_negative} ({actual_pct_negative:.1f}%)")
print(f"  Mixed/Neutral: {n_mixed} ({actual_pct_mixed:.1f}%)")
print(f"\\nComment Sentiment Score: {actual_sentiment:.3f}")
print(f"Engagement Velocity (estimated): {actual_velocity:.3f}")
print("=" * 70)

In [ ]:
# Compare predictions vs actual
print("\\n" + "=" * 70)
print("PREDICTED vs ACTUAL COMPARISON")
print("=" * 70)
print()

comparison_data = {
    'Metric': ['% Positive Reactions', '% Negative Reactions', 'Comment Sentiment', 'Engagement Velocity'],
    'Predicted': [pct_positive, pct_negative, comment_sentiment_dist, engagement_velocity],
    'Actual': [actual_pct_positive, actual_pct_negative, actual_sentiment, actual_velocity],
    'Difference': [
        pct_positive - actual_pct_positive,
        pct_negative - actual_pct_negative,
        comment_sentiment_dist - actual_sentiment,
        engagement_velocity - actual_velocity
    ]
}

comparison_df = pd.DataFrame(comparison_data)
comparison_df['Abs_Error'] = comparison_df['Difference'].abs()

print(comparison_df.to_string(index=False))
print()

# Calculate accuracy metrics
mae_positive = abs(pct_positive - actual_pct_positive)
mae_negative = abs(pct_negative - actual_pct_negative)
mae_sentiment = abs(comment_sentiment_dist - actual_sentiment)
mae_velocity = abs(engagement_velocity - actual_velocity)

print("\\nPrediction Accuracy:")
print(f"  % Positive Error: {mae_positive:.1f} percentage points")
print(f"  % Negative Error: {mae_negative:.1f} percentage points")
print(f"  Sentiment Error: {mae_sentiment:.3f}")
print(f"  Velocity Error: {mae_velocity:.3f}")

# Overall assessment
print("\\n" + "=" * 70)
print("ASSESSMENT")
print("=" * 70)

if mae_positive < 15 and mae_negative < 10:
    print("✓ Model predictions are reasonably close to actual results!")
elif mae_positive < 25 and mae_negative < 15:
    print("⚠ Model predictions are somewhat close, but there's room for improvement.")
else:
    print("✗ Model predictions differ significantly from actual results.")

print("\\nKey Observations:")
print(f"  • Model predicted {pct_positive:.1f}% positive, actual was {actual_pct_positive:.1f}%")
print(f"  • Model predicted {pct_negative:.1f}% negative, actual was {actual_pct_negative:.1f}%")
print(f"  • The post generated significant discussion and mixed reactions")
print(f"  • Some comments were highly polarized (very positive or very negative)")
print("=" * 70)

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Positive Reactions Comparison
ax1 = axes[0, 0]
categories = ['Predicted', 'Actual']
values = [pct_positive, actual_pct_positive]
colors = ['#3498db', '#2ecc71']
bars1 = ax1.bar(categories, values, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
ax1.set_ylabel('Percentage (%)', fontsize=12)
ax1.set_title('% Positive Reactions: Predicted vs Actual', fontsize=13, fontweight='bold')
ax1.set_ylim(0, max(values) * 1.2)
ax1.grid(axis='y', alpha=0.3)
for bar, val in zip(bars1, values):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             f'{val:.1f}%', ha='center', va='bottom', fontsize=11, fontweight='bold')

# 2. Negative Reactions Comparison
ax2 = axes[0, 1]
values2 = [pct_negative, actual_pct_negative]
colors2 = ['#3498db', '#e74c3c']
bars2 = ax2.bar(categories, values2, color=colors2, alpha=0.7, edgecolor='black', linewidth=1.5)
ax2.set_ylabel('Percentage (%)', fontsize=12)
ax2.set_title('% Negative Reactions: Predicted vs Actual', fontsize=13, fontweight='bold')
ax2.set_ylim(0, max(values2) * 1.2)
ax2.grid(axis='y', alpha=0.3)
for bar, val in zip(bars2, values2):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
             f'{val:.1f}%', ha='center', va='bottom', fontsize=11, fontweight='bold')

# 3. Sentiment Comparison
ax3 = axes[1, 0]
values3 = [comment_sentiment_dist, actual_sentiment]
bars3 = ax3.bar(categories, values3, color=['#3498db', '#9b59b6'], alpha=0.7, edgecolor='black', linewidth=1.5)
ax3.axhline(y=0, color='black', linestyle='--', linewidth=1, alpha=0.5)
ax3.set_ylabel('Sentiment Score', fontsize=12)
ax3.set_title('Comment Sentiment: Predicted vs Actual', fontsize=13, fontweight='bold')
ax3.set_ylim(-1, 1)
ax3.grid(axis='y', alpha=0.3)
for bar, val in zip(bars3, values3):
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height,
             f'{val:.3f}', ha='center', va='bottom' if val >= 0 else 'top', fontsize=11, fontweight='bold')

# 4. Velocity Comparison
ax4 = axes[1, 1]
values4 = [engagement_velocity, actual_velocity]
bars4 = ax4.bar(categories, values4, color=['#3498db', '#f39c12'], alpha=0.7, edgecolor='black', linewidth=1.5)
ax4.set_ylabel('Velocity Score', fontsize=12)
ax4.set_title('Engagement Velocity: Predicted vs Actual', fontsize=13, fontweight='bold')
ax4.set_ylim(0, 1)
ax4.grid(axis='y', alpha=0.3)
for bar, val in zip(bars4, values4):
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height,
             f'{val:.3f}', ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

### Insights from Actual Comments

**Key Themes in Comments:**

1. **Supportive Comments** focused on:
   - Understanding ambition and drive
   - Supporting founder culture
   - Recognizing different work styles
   - Appreciating the clarification about balance

2. **Critical Comments** raised concerns about:
   - Worker safety and burnout risks
   - Equity vs contractor compensation
   - Vulnerable employees (e.g., high school graduates)
   - Communication clarity (perception of backpedaling)

3. **Mixed/Neutral Comments** provided:
   - Balanced perspectives on founder vs employee expectations
   - Questions about the business model (contractors vs equity holders)
   - Recognition of timezone challenges in global companies

**Model Performance Notes:**
- The model predicted lower negative reactions (6.6%) than actual (26.1%)
- This suggests the model may underestimate controversy in posts that address work culture
- The sentiment prediction (0.381) was close to actual (0.348), showing good sentiment understanding
- The post generated significant discussion, which the model partially captured through engagement velocity

In [ ]:
# Visualize predictions
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Positive vs Negative Reactions
ax1 = axes[0, 0]
categories = ['Positive', 'Negative']
values = [pct_positive, pct_negative]
colors = ['#2ecc71', '#e74c3c']
bars = ax1.bar(categories, values, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
ax1.set_ylabel('Percentage (%)', fontsize=12)
ax1.set_title('Predicted Reaction Distribution', fontsize=14, fontweight='bold')
ax1.set_ylim(0, max(values) * 1.2)
ax1.grid(axis='y', alpha=0.3)

# Add value labels on bars
for bar, val in zip(bars, values):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             f'{val:.1f}%',
             ha='center', va='bottom', fontsize=11, fontweight='bold')

# 2. Comment Sentiment
ax2 = axes[0, 1]
sentiment_range = np.linspace(-1, 1, 100)
sentiment_colors = ['#e74c3c' if x < 0 else '#2ecc71' for x in sentiment_range]
ax2.barh([0], [comment_sentiment_dist], height=0.3, color='#3498db', alpha=0.7, edgecolor='black', linewidth=1.5)
ax2.axvline(x=0, color='black', linestyle='--', linewidth=1, alpha=0.5)
ax2.set_xlim(-1, 1)
ax2.set_xlabel('Sentiment Score', fontsize=12)
ax2.set_title('Comment Sentiment Distribution', fontsize=14, fontweight='bold')
ax2.set_yticks([])
ax2.text(comment_sentiment_dist, 0, f'{comment_sentiment_dist:.3f}',
         ha='center' if abs(comment_sentiment_dist) < 0.3 else ('left' if comment_sentiment_dist > 0 else 'right'),
         va='bottom', fontsize=11, fontweight='bold', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
ax2.grid(axis='x', alpha=0.3)

# 3. Engagement Velocity
ax3 = axes[1, 0]
velocity_range = np.linspace(0, 1, 100)
ax3.barh([0], [engagement_velocity], height=0.3, color='#9b59b6', alpha=0.7, edgecolor='black', linewidth=1.5)
ax3.set_xlim(0, 1)
ax3.set_xlabel('Velocity Score', fontsize=12)
ax3.set_title('Engagement Velocity', fontsize=14, fontweight='bold')
ax3.set_yticks([])
ax3.text(engagement_velocity, 0, f'{engagement_velocity:.3f}',
         ha='center', va='bottom', fontsize=11, fontweight='bold',
         bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
ax3.grid(axis='x', alpha=0.3)

# 4. Summary Metrics Table
ax4 = axes[1, 1]
ax4.axis('off')
table_data = [
    ['Metric', 'Value', 'Interpretation'],
    ['% Positive', f'{pct_positive:.1f}%', 'Higher is better'],
    ['% Negative', f'{pct_negative:.1f}%', 'Lower is better'],
    ['Sentiment', f'{comment_sentiment_dist:.3f}', 'Range: -1 to +1'],
    ['Velocity', f'{engagement_velocity:.3f}', 'Range: 0 to 1']
]
table = ax4.table(cellText=table_data, cellLoc='left', loc='center',
                  colWidths=[0.3, 0.25, 0.45])
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2)
# Style header row
for i in range(3):
    table[(0, i)].set_facecolor('#34495e')
    table[(0, i)].set_text_props(weight='bold', color='white')
# Style data rows
for i in range(1, 5):
    for j in range(3):
        if i % 2 == 0:
            table[(i, j)].set_facecolor('#ecf0f1')
ax4.set_title('Summary Metrics', fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
plt.show()

## Summary and Next Steps

### Model Performance Summary
- The model has been trained on {len(X_train)} training examples
- Test set performance metrics are shown above
- High-uncertainty predictions have been identified as potential failure modes

### Key Insights
1. **Feature Contributions**: Text embeddings capture semantic meaning, while metadata provides context
2. **Uncertainty**: The model surfaces uncertainty through error analysis
3. **Failure Modes**: High-uncertainty predictions indicate where the model is less confident

### Recommendations
- Collect more training data, especially for edge cases
- Experiment with different embedding models (e.g., gemini-embedding-001)
- Tune hyperparameters further using cross-validation
- Consider ensemble methods for improved robustness
- Monitor model performance on new data and retrain periodically